# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io

path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)


# Constant

In [2]:
CURRENT_DATE = datetime.now().strftime("%Y%m%d")


# Config

In [3]:
unique_id = str(uuid.uuid4()).replace('-', '')[-12:]
print(f"The unique Id is: {unique_id}")
BUCKETNAME = 'prod-asia-southeast1-tonik-aiml-workspace'
CLOUDPATH = 'DC/Model_Monitoring/Model_Tables'
LOCALPATH = r'D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Data Engineering\Model_Monitoring\New_Model_Monitoring\Data'
VERSION = 'V1'

The unique Id is: 134933f53c90


# <div align="left" style="color:rgb(51, 250, 250);"> Functions </div>

## <div align="left" style="color:rgb(51, 250, 250);"> Save the data to google clound storage </div>

In [4]:
def save_df_to_gcs(df, bucket_name, destination_blob_name, file_format='csv'):
    """Saves a pandas DataFrame to Google Cloud Storage.

    Args:
        df: The pandas DataFrame to save.
        bucket_name: The name of the GCS bucket.
        destination_blob_name: The name of the blob to be created.
        file_format: The file format to save the DataFrame in ('csv' or 'parquet').
    """

    # Create a temporary file
    if file_format == 'csv':
        temp_file = 'temp.csv'
        df.to_csv(temp_file, index=False)
    elif file_format == 'parquet':
        temp_file = 'temp.parquet'
        df.to_parquet(temp_file, index=False)
    else:
        raise ValueError("Invalid file format. Please choose 'csv' or 'parquet'.")

    # Upload the file to GCS
    storage_client = storage.Client(project="prj-prod-dataplatform")

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(temp_file)

    # Remove the temporary file
    import os
    os.remove(temp_file)
    


## <div align="left" style="color:rgb(51, 250, 250);"> Read the Data from Google Cloud Storage </div>

In [5]:
def read_df_from_gcs(bucket_name, source_blob_name, file_format='csv'):
    """Reads a DataFrame from Google Cloud Storage.

    Args:
        bucket_name: The name of the GCS bucket.
        source_blob_name: The name of the blob to read.
        file_format: The file format to read ('csv' or 'parquet').

    Returns:
        pandas.DataFrame: The data loaded from the GCS file.
    """
    # Create a temporary file name
    temp_file = f'temp.{file_format}'
    
    try:
        # Initialize GCS client
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)

        # Download the file to a temporary location
        blob.download_to_filename(temp_file)

        # Read the file into a DataFrame
        if file_format == 'csv':
            df = pd.read_csv(temp_file, low_memory=False)
        elif file_format == 'parquet':
            df = pd.read_parquet(temp_file)
        else:
            raise ValueError("Invalid file format. Please choose 'csv' or 'parquet'.")

        return df

    finally:
        # Clean up the temporary file
        if os.path.exists(temp_file):
            os.remove(temp_file)

## <div align = "left" style="color:rgb(51, 250, 250);"> Data Quality Report </div>

In [6]:
def data_quality_report(df, target_col='ln_fspd30_flag'):
    # Initialize an empty list to store each row of data
    report_data = []
    # Iterate over each column in the DataFrame to compute metrics
    for col in df.columns:
        # Determine the data type of the column
        data_type = df[col].dtype
       
        # Calculate the number of missing values in the column
        missing_values = df[col].isnull().sum()
       
        # Calculate the percentage of missing values relative to the total number of rows
        missing_percentage = (missing_values / len(df)) * 100
       
        # Calculate the number of unique values in the column
        unique_values = df[col].nunique()
       
        # Calculate the percentage of non-missing values
        non_missing_percentage = ((len(df) - missing_values) / len(df)) * 100
       
        # Check if the column is numeric to compute additional metrics
        if pd.api.types.is_numeric_dtype(df[col]):
            # Compute minimum, maximum, mean, median, mode, mode percentage, standard deviation, and quantiles
            min_value = df[col].min()
            max_value = df[col].max()
            mean_value = df[col].mean()
            median_value = df[col].median()
            mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else None
            mode_percentage = (df[col] == mode_value).sum() / len(df) * 100 if mode_value is not None else None
            std_dev = df[col].std()
            quantile_25 = df[col].quantile(0.25)
            quantile_50 = df[col].quantile(0.50)  # Same as median
            quantile_75 = df[col].quantile(0.75)
            
            # Calculate the Interquartile Range (IQR)
            iqr = quantile_75 - quantile_25
            
            # Calculate Skewness and Kurtosis
            skewness = df[col].skew()
            kurtosis = df[col].kurt()
            
            # Calculate Coefficient of Variation (CV) - standardized measure of dispersion
            cv = (std_dev / mean_value) * 100 if mean_value != 0 else None
            
            # Calculate correlation with target variable if target exists in dataframe
            if target_col in df.columns and col != target_col and pd.api.types.is_numeric_dtype(df[target_col]):
                # Calculate correlation only using rows where both columns have non-null values
                correlation = df[[col, target_col]].dropna().corr().iloc[0, 1]
            else:
                correlation = None
        else:
            # Assign None for non-numeric columns where appropriate
            min_value = None
            max_value = None
            mean_value = None
            median_value = None
            mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else None
            mode_percentage = (df[col] == mode_value).sum() / len(df) * 100 if mode_value is not None else None
            std_dev = None
            quantile_25 = None
            quantile_50 = None
            quantile_75 = None
            iqr = None
            skewness = None
            kurtosis = None
            cv = None
            correlation = None
       
        # Append the computed metrics for the current column to the list
        report_data.append({
            'Column': col,
            'Data Type': data_type,
            'Missing Values': missing_values,
            'Missing Percentage': missing_percentage,
            'Unique Values': unique_values,
            'Min': min_value,
            'Max': max_value,
            'Mean': mean_value,
            'Median': median_value,
            'Mode': mode_value,
            'Mode Percentage': mode_percentage,
            'Std Dev': std_dev,
            'Non-missing Percentage': non_missing_percentage,
            '25% Quantile': quantile_25,
            '50% Quantile': quantile_50,
            '75% Quantile': quantile_75,
            'IQR': iqr,
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'CV (%)': cv,
            f'Correlation with {target_col}': correlation
        })
    # Create the DataFrame from the list of dictionaries
    report = pd.DataFrame(report_data)
   
    # Return the complete data quality report DataFrame
    return report

# <div align = "left" style="color:rgb(51,250,250);"> Upload pickle file to Google Cloud Storage Bucke </div>

In [7]:
def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to Google Cloud Storage"""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    blob.upload_from_filename(source_file_path)
    print(f"File {source_file_path} uploaded to {bucket_name}/{destination_blob_name}")

In [8]:
import pickle
import io
from google.cloud import storage
def save_pickle_to_gcs(data, bucket_name, destination_blob_name):
    """
    Save any Python object as a pickle file to Google Cloud Storage
    
    Args:
        data: The Python object to pickle (DataFrame, dict, list, etc.)
        bucket_name: Name of the GCS bucket
        destination_blob_name: Path/filename in the bucket
    """
    # Initialize the GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    # Serialize the data to pickle format in memory
    pickle_buffer = io.BytesIO()
    pickle.dump(data, pickle_buffer)
    pickle_buffer.seek(0)
    
    # Upload the pickle data to GCS
    blob.upload_from_file(pickle_buffer, content_type='application/octet-stream')
    print(f"Pickle file uploaded to gs://{bucket_name}/{destination_blob_name}")

# save_dataframe_multi_format

In [9]:
def save_dataframe_multi_format(
    dataframe: pd.DataFrame, 
    cloud_path: str, 
    filename: str, 
    client: bigquery.Client = None,
    bucket_name: str = None
) -> dict:
    """
    Save a pandas DataFrame to Google Cloud Storage in multiple formats (CSV, Pickle, Parquet, Joblib).
    
    Args:
        dataframe (pd.DataFrame): The DataFrame to save
        cloud_path (str): The cloud path (e.g., 'DC/Model_Monitoring/cash_beta_trench1_data')
        filename (str): The base filename without extension
        client (bigquery.Client, optional): BigQuery client (for project reference)
        bucket_name (str, optional): GCS bucket name. If None, will try to extract from client
        
    Returns:
        dict: Dictionary with status of each file saved
        
    Example:
        client = bigquery.Client(project='prj-prod-dataplatform')
        CLOUDPATH = 'DC/Model_Monitoring/cash_beta_trench1_data'
        
        results = save_dataframe_multi_format(
            dataframe=d1,
            cloud_path=CLOUDPATH,
            filename='my_data',
            client=client,
            bucket_name='your-bucket-name'  # Replace with your actual bucket name
        )
    """
    
    # Initialize Google Cloud Storage client
    storage_client = storage.Client(project=client.project if client else None)
    
    # You'll need to specify your bucket name here
    # Common bucket names in GCP data platforms might be like:
    # - 'prj-prod-dataplatform-storage'
    # - 'dataplatform-storage'
    # - or similar pattern
    if bucket_name is None:
        # You need to replace this with your actual bucket name
        raise ValueError("Please provide the bucket_name parameter")
    
    bucket = storage_client.bucket(bucket_name)
    
    # Results dictionary to track saves
    results = {}
    
    # Ensure cloud_path doesn't start with '/'
    cloud_path = cloud_path.lstrip('/')
    
    try:
        # 1. Save as CSV
        csv_buffer = io.StringIO()
        dataframe.to_csv(csv_buffer, index=False)
        csv_blob = bucket.blob(f"{cloud_path}/{filename}.csv")
        csv_blob.upload_from_string(csv_buffer.getvalue(), content_type='text/csv')
        results['csv'] = f"gs://{bucket_name}/{cloud_path}/{filename}.csv"
        
        # 2. Save as Pickle
        pickle_buffer = io.BytesIO()
        pickle.dump(dataframe, pickle_buffer)
        pickle_blob = bucket.blob(f"{cloud_path}/{filename}.pkl")
        pickle_blob.upload_from_string(pickle_buffer.getvalue(), content_type='application/octet-stream')
        results['pickle'] = f"gs://{bucket_name}/{cloud_path}/{filename}.pkl"
        
        # 3. Save as Parquet
        parquet_buffer = io.BytesIO()
        dataframe.to_parquet(parquet_buffer, index=False)
        parquet_blob = bucket.blob(f"{cloud_path}/{filename}.parquet")
        parquet_blob.upload_from_string(parquet_buffer.getvalue(), content_type='application/octet-stream')
        results['parquet'] = f"gs://{bucket_name}/{cloud_path}/{filename}.parquet"
        
        # 4. Save as Joblib
        joblib_buffer = io.BytesIO()
        joblib.dump(dataframe, joblib_buffer)
        joblib_blob = bucket.blob(f"{cloud_path}/{filename}.joblib")
        joblib_blob.upload_from_string(joblib_buffer.getvalue(), content_type='application/octet-stream')
        results['joblib'] = f"gs://{bucket_name}/{cloud_path}/{filename}.joblib"
        
        print("All files saved successfully!")
        for format_type, path in results.items():
            print(f"{format_type.upper()}: {path}")
            
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        results['error'] = str(e)
    
    return results

# cash_beta_trench1_applied_loans_backscored_20241001_20250831

# Table

In [10]:
schema1 = 'worktable_data_analysis'
cash_beta_trench1 = f'cash_beta_trench1_applied_loans_backscored_20241001_20250831'

# Query

In [11]:
sq = f"""
select * from worktable_data_analysis.cash_beta_trench1_applied_loans_backscored_20241001_20250831;
"""
d1 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of {schema1}.{cash_beta_trench1} table is:\t {d1.shape}")

Job ID 15aba8a8-af71-4896-a675-0ab74862e7d4 successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of worktable_data_analysis.cash_beta_trench1_applied_loans_backscored_20241001_20250831 table is:	 (296480, 34)


In [12]:
d1.head()

,customer_id,digitalLoanAccountId,days_on_book,ln_appln_submit_datetime,ln_os_type,ln_vas_opted_flag,ln_self_dec_income,ln_age,ln_source_funds_new_bin,ln_loan_level_user_type,ln_industry_new_cat_bin,ln_marital_status,ln_doc_type_rolled,ln_education_level,ln_ref2_type,ln_email_primary_domain,ln_province_bin,ln_mature_fspd30_flag,ln_fspd30_flag,demo_score,trench_category,ln_loan_type,ln_disb_dtime,c_app_cnt_health_and_fitness_ever,c_app_cnt_shopping_ever,c_app_median_time_bw_installed_mins_ever,c_app_avg_time_bw_installed_mins_3d,c_app_cnt_crypto_ever,c_app_cnt_driver_ever,c_app_cnt_payday_180d,c_app_cnt_gambling_180d,apps_score,credo_score,stack_score
0,3461645,6061f751-bb86-49a3-9b6b-0ea44b4306d6,0,2025-05-27 13:22:29+00:00,Android,1,28500,17,salary,2_New Applicant,1.0,Single,national id,College Undergraduate,Friend,gmail.com,others,<NA>,<NA>,0.624579,Trench 1,Quick,NaT,0.0,2.0,0.033333,NaN,0.0,0.0,0.0,0.0,0.642715,0.240666,0.895453
1,3292475,80468034-cc21-4da8-b2aa-6e35c84ac387,1,2025-02-28 13:24:31+00:00,Android,1,15000,17,income from business,2_New Applicant,1.0,Single,national id,College Graduate,Sibling,gmail.com,others,<NA>,<NA>,0.680918,Trench 1,Quick,NaT,2.0,0.0,1499.500000,1136.783333,0.0,0.0,3.0,0.0,0.532071,0.393578,0.697355
2,3192798,03cf7323-cf3a-44be-9975-ac302d61e7ed,0,2025-01-16 03:58:30+00:00,iOS,1,50000,18,salary,2_New Applicant,2.0,Single,driving license,College Graduate,Sibling,gmail.com,others,<NA>,<NA>,0.587765,Trench 1,Quick,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.220727,0.610448
3,3573079,0e938f32-94e3-4473-9beb-9469807b1edf,0,2025-07-23 08:23:10+00:00,iOS,1,6000,18,remittance,2_New Applicant,missing,Single,national id,College Graduate,Sibling,gmail.com,others,<NA>,<NA>,0.660842,Trench 1,Quick,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.243710,0.705892
4,3631270,3eb7e8c9-1412-46b2-bdfe-aa71f1ac9a14,0,2025-08-19 18:56:40+00:00,iOS,1,5000,18,income from business,2_New Applicant,1.0,Single,national id,College Undergraduate,Friend,gmail.com,others,<NA>,<NA>,0.701313,Trench 1,Quick,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.280322,0.770477


In [13]:
d1.columns.values

array(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d', 'apps_score', 'credo_score',
       'stack_score'], dtype=object)

In [14]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag',  'trench_category', 'ln_loan_type', 'ln_disb_dtime',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('demo_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench1_Demo_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [15]:
transformeddata = f'cash_beta_trench1_demo'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the transformed data is: {dfd.shape}")

The shape of the transformed data is: (296480, 13)


In [16]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,3461645,6061f751-bb86-49a3-9b6b-0ea44b4306d6,1d75e484-e659-4276-b57e-90b92275f2c6,0.624579,2025-09-18T08:50:53.563107,2025-09-18T08:50:53.563107,Cash_beta_trench1_Demo_backscore,v1,trench alpha beta,c40b9e39-01e7-420b-8d58-9842f99e1144,2025-09-18T08:50:53.563107,{},"{""days_on_book"": 0, ""ln_appln_submit_datetime""..."
1,3292475,80468034-cc21-4da8-b2aa-6e35c84ac387,ece957c1-426e-49f4-990b-8528427483b4,0.680918,2025-09-18T08:50:53.563107,2025-09-18T08:50:53.563107,Cash_beta_trench1_Demo_backscore,v1,trench alpha beta,1a7cb064-1911-4dc3-a499-beb42ef01a51,2025-09-18T08:50:53.563107,{},"{""days_on_book"": 1, ""ln_appln_submit_datetime""..."
2,3192798,03cf7323-cf3a-44be-9975-ac302d61e7ed,599ab9e8-d706-4be8-a032-0157045ff487,0.587765,2025-09-18T08:50:53.564117,2025-09-18T08:50:53.564117,Cash_beta_trench1_Demo_backscore,v1,trench alpha beta,35d148fc-b0dd-4a19-a239-11c8a3aaf657,2025-09-18T08:50:53.564117,{},"{""days_on_book"": 0, ""ln_appln_submit_datetime""..."
3,3573079,0e938f32-94e3-4473-9beb-9469807b1edf,44416b51-c2eb-4924-87a6-30d80c95b9b3,0.660842,2025-09-18T08:50:53.564117,2025-09-18T08:50:53.564117,Cash_beta_trench1_Demo_backscore,v1,trench alpha beta,18e653cf-d136-49be-a9b3-e7600b60eecf,2025-09-18T08:50:53.564117,{},"{""days_on_book"": 0, ""ln_appln_submit_datetime""..."
4,3631270,3eb7e8c9-1412-46b2-bdfe-aa71f1ac9a14,dccb8d37-c25f-45a7-bd65-15888091e539,0.701313,2025-09-18T08:50:53.564117,2025-09-18T08:50:53.564117,Cash_beta_trench1_Demo_backscore,v1,trench alpha beta,01d28bf3-48e6-4fa7-8384-f1c0ee27c0fd,2025-09-18T08:50:53.564117,{},"{""days_on_book"": 0, ""ln_appln_submit_datetime""..."


In [17]:
dfd.columns

Index(['customerId', 'digitalLoanAccountId', 'crifApplicationId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'subscription_name', 'message_id', 'publish_time', 'attributes',
       'calcFeature'],
      dtype='object')

In [18]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [19]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench1_demo
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_demo.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_demo.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_demo.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_demo.joblib


# Insert into a table

In [20]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=44a67f0b-d79b-4c14-b21f-a971250287ad>

In [21]:
d2 = d1[d1['ln_os_type'] == 'Android']

In [22]:
d2.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d', 'apps_score', 'credo_score', 'stack_score'],
      dtype='object')

In [23]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d2.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d'
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('apps_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench1_appscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [24]:
transformeddata = f'cash_beta_trench1_app'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the transformed data is: {dfd.shape}")

The shape of the transformed data is: (201845, 13)


In [25]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,3461645,6061f751-bb86-49a3-9b6b-0ea44b4306d6,90bde26a-ab8d-4f62-9c3c-52e7d17b0aa6,0.642715,2025-09-18T08:53:15.131546,2025-09-18T08:53:15.131546,Cash_beta_trench1_appscore_backscore,v1,trench alpha beta,1b1df99e-0437-4b51-8397-1145d17f7dbd,2025-09-18T08:53:15.131546,{},"{""c_app_cnt_health_and_fitness_ever"": 0.0, ""c_..."
1,3292475,80468034-cc21-4da8-b2aa-6e35c84ac387,ae4186c6-616b-4f16-94dc-1fdcde38f83d,0.532071,2025-09-18T08:53:15.131546,2025-09-18T08:53:15.131546,Cash_beta_trench1_appscore_backscore,v1,trench alpha beta,d69a4245-aad7-48ab-801c-f4682bced262,2025-09-18T08:53:15.131546,{},"{""c_app_cnt_health_and_fitness_ever"": 2.0, ""c_..."
2,2943376,38939b31-211f-456e-b9c1-9ace4f431b3a,82ff696a-427e-47b9-a01b-03d885892203,0.732883,2025-09-18T08:53:15.131546,2025-09-18T08:53:15.131546,Cash_beta_trench1_appscore_backscore,v1,trench alpha beta,f92298f5-a562-41a5-91e7-2cd166329d6a,2025-09-18T08:53:15.131546,{},"{""c_app_cnt_health_and_fitness_ever"": 0.0, ""c_..."
3,3076048,c7608431-0327-4448-aaf8-dd1f244aa0ba,d1dadb8e-bafe-4d1f-8560-a31698ff01ec,0.648517,2025-09-18T08:53:15.131546,2025-09-18T08:53:15.131546,Cash_beta_trench1_appscore_backscore,v1,trench alpha beta,0b6bd531-8998-4807-a504-b479c9582033,2025-09-18T08:53:15.131546,{},"{""c_app_cnt_health_and_fitness_ever"": 0.0, ""c_..."
4,3648485,7af56b05-af84-4460-b83e-e11ab6c77e81,670303bf-9d38-4a42-8840-fa2ef52273ee,0.538931,2025-09-18T08:53:15.132547,2025-09-18T08:53:15.132547,Cash_beta_trench1_appscore_backscore,v1,trench alpha beta,833b124d-2dc6-47a4-95b9-f3e599a16602,2025-09-18T08:53:15.132547,{},"{""c_app_cnt_health_and_fitness_ever"": 0.0, ""c_..."


In [26]:
dfd.columns

Index(['customerId', 'digitalLoanAccountId', 'crifApplicationId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'subscription_name', 'message_id', 'publish_time', 'attributes',
       'calcFeature'],
      dtype='object')

In [27]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

Found no duplicate digitalLoanAccountId

In [28]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench1_app
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_app.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_app.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_app.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_app.joblib


# Insert into a table

In [29]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e8b9d494-e695-4d06-a472-43d808c27610>

In [30]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d', 'apps_score', 'credo_score', 'stack_score'],
      dtype='object')

In [31]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
         'demo_score', 'apps_score', 'credo_score',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('stack_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench1_stackscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [32]:
transformeddata = f'cash_beta_trench1_stackscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the transformed data is: {dfd.shape}")

The shape of the transformed data is: (296480, 13)


In [33]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,3461645,6061f751-bb86-49a3-9b6b-0ea44b4306d6,bb5e49eb-2b65-46d5-80df-1256a8a6eaae,0.895453,2025-09-18T08:54:11.019090,2025-09-18T08:54:11.019090,Cash_beta_trench1_stackscore_backscore,v1,trench alpha beta,583bfba7-5e63-4988-9798-a83a593cc6d1,2025-09-18T08:54:11.019090,{},"{""demo_score"": 0.6245786265216958, ""apps_score..."
1,3292475,80468034-cc21-4da8-b2aa-6e35c84ac387,c2512a6f-9239-4d7f-9130-82989b4c7b97,0.697355,2025-09-18T08:54:11.020091,2025-09-18T08:54:11.020091,Cash_beta_trench1_stackscore_backscore,v1,trench alpha beta,5a52607a-19e1-4c7a-a3c4-6d5b3bba5695,2025-09-18T08:54:11.020091,{},"{""demo_score"": 0.6809183034639812, ""apps_score..."
2,3192798,03cf7323-cf3a-44be-9975-ac302d61e7ed,668990f4-650c-4395-ad18-19ff8ac9bc4b,0.610448,2025-09-18T08:54:11.020091,2025-09-18T08:54:11.020091,Cash_beta_trench1_stackscore_backscore,v1,trench alpha beta,b6ef8c31-4172-4318-bb5b-9033945f0153,2025-09-18T08:54:11.020091,{},"{""demo_score"": 0.5877648931665984, ""credo_scor..."
3,3573079,0e938f32-94e3-4473-9beb-9469807b1edf,b043fd3c-6fd8-4b88-9fe4-12d75ff51294,0.705892,2025-09-18T08:54:11.020091,2025-09-18T08:54:11.020091,Cash_beta_trench1_stackscore_backscore,v1,trench alpha beta,b6e973b0-8b51-4810-8ea6-039c08fb054a,2025-09-18T08:54:11.020091,{},"{""demo_score"": 0.6608418447966299, ""credo_scor..."
4,3631270,3eb7e8c9-1412-46b2-bdfe-aa71f1ac9a14,d21c0631-0139-4fa7-81da-1ff8f004b45c,0.770477,2025-09-18T08:54:11.020091,2025-09-18T08:54:11.020091,Cash_beta_trench1_stackscore_backscore,v1,trench alpha beta,e3418118-b8c5-48c8-9b81-87b9c0e5046b,2025-09-18T08:54:11.020091,{},"{""demo_score"": 0.7013128224385257, ""credo_scor..."


In [34]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [35]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench1_stackscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_stackscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_stackscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_stackscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench1_stackscore.joblib


# Insert into a table

In [36]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=233f8664-a46b-4b06-a9e0-7742e77b2d14>

# cash_beta_trench2_applied_loans_backscored_20241001_20250831

# Table

In [37]:
schema1 = 'worktable_data_analysis'
cash_beta_trench2 = f'cash_beta_trench2_applied_loans_backscored_20241001_20250831'

# Query

In [38]:
sq = f"""
select * from {schema1}.{cash_beta_trench2};
"""
d1 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of {schema1}.{cash_beta_trench2} table is:\t {d1.shape}")

Job ID e2230f99-e611-4011-a3f8-5d6c556cd6e2 successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of worktable_data_analysis.cash_beta_trench2_applied_loans_backscored_20241001_20250831 table is:	 (111973, 46)


In [39]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d', 'c_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'c_t2_tx_first_product_user_segment_WOE',
       'c_t2_tx_first_applied_loan_type_bin_WOE', 'c_t2_tx_cnt_rejected_loans',
       'c_t2_tx_apps

In [40]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
         'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category', 'ln_loan_type', 'ln_disb_dtime',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('demo_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench2_demo_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [41]:
transformeddata = f'cash_beta_trench2_demoscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the transformed data is: {dfd.shape}")

The shape of the transformed data is: (111973, 13)


In [42]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2292294,76c329f1-a80c-4f97-905a-cdc50eaefc4f,82388715-8aba-4db1-8dd6-65c2ea929800,0.558904,2025-09-18T08:56:10.942261,2025-09-18T08:56:10.942261,Cash_beta_trench2_demo_backscore,v1,trench alpha beta,065765d5-0382-4e6b-bc98-95451d57ae27,2025-09-18T08:56:10.942261,{},"{""days_on_book"": 603, ""ln_appln_submit_datetim..."
1,3109464,2f55c1fd-edde-4d76-804c-9faa6fe8276f,2b4f5221-7a0e-41a8-b48c-b46e8ed39912,0.599236,2025-09-18T08:56:10.942261,2025-09-18T08:56:10.942261,Cash_beta_trench2_demo_backscore,v1,trench alpha beta,cb63712b-e055-42ed-8fa2-1f430a780425,2025-09-18T08:56:10.942261,{},"{""days_on_book"": 112, ""ln_appln_submit_datetim..."
2,1491167,32ff1b2c-d847-4b05-8636-9c8e4aeba209,04eb54b3-6978-41f5-800a-54534ae16759,0.498566,2025-09-18T08:56:10.942261,2025-09-18T08:56:10.942261,Cash_beta_trench2_demo_backscore,v1,trench alpha beta,833bde18-4e23-4311-b23c-78cba43668f7,2025-09-18T08:56:10.942261,{},"{""days_on_book"": 846, ""ln_appln_submit_datetim..."
3,2305465,9715ee51-7fac-4eda-b675-310d6ab79ac8,b8c71f5f-346b-49c8-8e24-d941fadca0fb,0.346935,2025-09-18T08:56:10.942261,2025-09-18T08:56:10.942261,Cash_beta_trench2_demo_backscore,v1,trench alpha beta,cd715a77-6ab4-40a7-bde7-3c5b8de43e97,2025-09-18T08:56:10.942261,{},"{""days_on_book"": 351, ""ln_appln_submit_datetim..."
4,2080167,567e6cf6-9f0f-4fb8-8ee5-1d0022d34d2a,07282fee-c74d-47c8-938e-d355443be0c1,0.611919,2025-09-18T08:56:10.942261,2025-09-18T08:56:10.942261,Cash_beta_trench2_demo_backscore,v1,trench alpha beta,ea367251-b047-4d1b-96be-50b1c3f402f5,2025-09-18T08:56:10.942261,{},"{""days_on_book"": 492, ""ln_appln_submit_datetim..."


In [43]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [44]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench2_demoscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_demoscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_demoscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_demoscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_demoscore.joblib


# Insert into a table

In [45]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7f40e898-a847-40b1-8f5e-86e6b10bdc69>

In [46]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d', 'c_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'c_t2_tx_first_product_user_segment_WOE',
       'c_t2_tx_first_applied_loan_type_bin_WOE', 'c_t2_tx_cnt_rejected_loans',
       'c_t2_tx_apps

In [47]:
d2 = d1[d1['ln_os_type']=='Android'].copy()
d2['ln_os_type'].value_counts()

ln_os_type
Android    74603
Name: count, dtype: int64

In [48]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d2.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
         'ln_os_type', 'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('apps_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench2_appscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [49]:
transformeddata = f'cash_beta_trench2_appscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the transformed data is: {dfd.shape}")

The shape of the transformed data is: (74603, 13)


In [50]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2904485,d1983361-f30d-4594-be95-d4b45977a87a,6c5dc11c-2f2b-462e-a0a3-4094ebc302a2,0.462671,2025-09-18T08:56:53.136898,2025-09-18T08:56:53.136898,Cash_beta_trench2_appscore_backscore,v1,trench alpha beta,177454f9-42c6-4273-a0a4-bfb3bb8090d9,2025-09-18T08:56:53.136898,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_health_an..."
1,2532401,a87a15ae-3400-4967-9842-9473ef8018e4,1a12557d-67e5-4008-b9d5-9e2ac90c9c5a,0.524205,2025-09-18T08:56:53.136898,2025-09-18T08:56:53.136898,Cash_beta_trench2_appscore_backscore,v1,trench alpha beta,78621c6e-879a-4a50-9451-b156244d2224,2025-09-18T08:56:53.136898,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_health_an..."
2,1193953,c8877602-9d57-4926-b92f-99ace007dc69,df4265c6-9a18-4688-9cdd-1cbc501a79af,0.495497,2025-09-18T08:56:53.137899,2025-09-18T08:56:53.137899,Cash_beta_trench2_appscore_backscore,v1,trench alpha beta,5d69b97c-8157-404d-a421-ee2dd300899e,2025-09-18T08:56:53.137899,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_health_an..."
3,3011213,b4c2f0d9-6d75-4f47-abb1-1c5aa03fa6c6,e52174a4-447d-485e-950a-b54fff764e34,0.559838,2025-09-18T08:56:53.137899,2025-09-18T08:56:53.137899,Cash_beta_trench2_appscore_backscore,v1,trench alpha beta,a2f1559b-d96e-4d18-84ad-1be9d972dc2e,2025-09-18T08:56:53.137899,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_health_an..."
4,2912260,5e3fac60-ef9c-49e5-9b1b-7680e9f3b711,95ea3f6e-9b92-4965-a534-6f023fd235f9,0.601748,2025-09-18T08:56:53.137899,2025-09-18T08:56:53.137899,Cash_beta_trench2_appscore_backscore,v1,trench alpha beta,2cb75ce1-170b-4820-b453-2bea2714936f,2025-09-18T08:56:53.137899,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_health_an..."


In [51]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [52]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench2_appscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_appscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_appscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_appscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_appscore.joblib


# Insert into a table

In [53]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6090479a-ac35-42c4-bc5f-4f07f4a75d98>

In [54]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d', 'c_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'c_t2_tx_first_product_user_segment_WOE',
       'c_t2_tx_first_applied_loan_type_bin_WOE', 'c_t2_tx_cnt_rejected_loans',
       'c_t2_tx_apps

In [55]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
         'c_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'c_t2_tx_first_product_user_segment_WOE',
       'c_t2_tx_first_applied_loan_type_bin_WOE', 'c_t2_tx_cnt_rejected_loans',
       'c_t2_tx_appsflyer_install_to_registration_minutes',
       'c_t2_tx_first_applied_loan_amount', 'c_t2_tx_deposit_accnt_cnt',
       'c_t2_tx_cnt_cash_in_total', 'c_t2_tx_cnt_incomplete_loan_apps',
       'c_t2_tx_amt_cash_in_total', 'c_t2_tx_last_applied_loan_tenor_bin_WOE',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('trx_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench2_transactionscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [56]:
transformeddata = f'cash_beta_trench2_transactioncore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the cash_beta_trench2_transactioncore data is: (111973, 13)


In [57]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2292294,76c329f1-a80c-4f97-905a-cdc50eaefc4f,61ea3cf9-3974-43c4-ab9b-998d737f9bcf,0.550381,2025-09-18T08:57:20.158546,2025-09-18T08:57:20.158546,Cash_beta_trench2_transactionscore_backscore,v1,trench alpha beta,717b0d06-656a-4240-b7a5-130c8a807b32,2025-09-18T08:57:20.158546,{},"{""c_t2_tx_meng_ql_calculator_tot_visit_cnt"": 3..."
1,3109464,2f55c1fd-edde-4d76-804c-9faa6fe8276f,ae7a93c2-7c0f-44ad-8924-7dd0ea454a84,0.531987,2025-09-18T08:57:20.158546,2025-09-18T08:57:20.158546,Cash_beta_trench2_transactionscore_backscore,v1,trench alpha beta,272bd9d3-59e8-433e-b71c-3689a407c8f8,2025-09-18T08:57:20.158546,{},"{""c_t2_tx_meng_ql_calculator_tot_visit_cnt"": 1..."
2,1491167,32ff1b2c-d847-4b05-8636-9c8e4aeba209,6014b9cd-cbc0-49b4-907d-9aa195f021f1,0.560202,2025-09-18T08:57:20.158546,2025-09-18T08:57:20.158546,Cash_beta_trench2_transactionscore_backscore,v1,trench alpha beta,dbe76832-8b33-42b6-b9e1-8ff7becdb696,2025-09-18T08:57:20.158546,{},"{""c_t2_tx_meng_ql_calculator_tot_visit_cnt"": 3..."
3,2305465,9715ee51-7fac-4eda-b675-310d6ab79ac8,a3a36232-ff21-4de6-ac19-059ded52aad9,0.515623,2025-09-18T08:57:20.158546,2025-09-18T08:57:20.158546,Cash_beta_trench2_transactionscore_backscore,v1,trench alpha beta,87fde8d6-2141-406e-a6c7-ea1a77a89194,2025-09-18T08:57:20.158546,{},"{""c_t2_tx_meng_ql_calculator_tot_visit_cnt"": 1..."
4,2080167,567e6cf6-9f0f-4fb8-8ee5-1d0022d34d2a,01f35f68-e317-4448-9a43-9f6d87db858a,0.452746,2025-09-18T08:57:20.159546,2025-09-18T08:57:20.159546,Cash_beta_trench2_transactionscore_backscore,v1,trench alpha beta,445e44e1-01fd-4d25-bd89-40cfa5577d42,2025-09-18T08:57:20.159546,{},"{""c_t2_tx_meng_ql_calculator_tot_visit_cnt"": 1..."


In [58]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [59]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench2_transactioncore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_transactioncore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_transactioncore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_transactioncore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_transactioncore.joblib


# Insert into a table

In [60]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=60819143-618e-4529-b057-97d8983c206a>

In [61]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_health_and_fitness_ever', 'c_app_cnt_shopping_ever',
       'c_app_median_time_bw_installed_mins_ever',
       'c_app_avg_time_bw_installed_mins_3d', 'c_app_cnt_crypto_ever',
       'c_app_cnt_driver_ever', 'c_app_cnt_payday_180d',
       'c_app_cnt_gambling_180d', 'c_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'c_t2_tx_first_product_user_segment_WOE',
       'c_t2_tx_first_applied_loan_type_bin_WOE', 'c_t2_tx_cnt_rejected_loans',
       'c_t2_tx_apps

In [62]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
          'demo_score', 'trx_score', 'apps_score', 'credo_score',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('stack_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench2_stackscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [63]:
transformeddata = f'cash_beta_trench2_stackcore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the cash_beta_trench2_stackcore data is: (111973, 13)


In [64]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2292294,76c329f1-a80c-4f97-905a-cdc50eaefc4f,a9e1331e-b1b7-478c-a6cd-02a36c3b442d,0.621407,2025-09-18T08:58:01.752673,2025-09-18T08:58:01.752673,Cash_beta_trench2_stackscore_backscore,v1,trench alpha beta,36f96f09-313a-4b54-b4e3-510779f23a5c,2025-09-18T08:58:01.752673,{},"{""demo_score"": 0.5589038824077822, ""trx_score""..."
1,3109464,2f55c1fd-edde-4d76-804c-9faa6fe8276f,51752e3b-fc88-45b5-a11b-e72ba3d13fa5,0.519240,2025-09-18T08:58:01.753671,2025-09-18T08:58:01.753671,Cash_beta_trench2_stackscore_backscore,v1,trench alpha beta,e62e4249-3a62-49dd-97a9-83deb7e693d0,2025-09-18T08:58:01.753671,{},"{""demo_score"": 0.599235865559278, ""trx_score"":..."
2,1491167,32ff1b2c-d847-4b05-8636-9c8e4aeba209,ff25ba95-4576-4b2e-b08e-7a7b6ba10979,0.452883,2025-09-18T08:58:01.753671,2025-09-18T08:58:01.753671,Cash_beta_trench2_stackscore_backscore,v1,trench alpha beta,5f7f91ac-2d23-405f-ade6-2ae397202ab0,2025-09-18T08:58:01.753671,{},"{""demo_score"": 0.49856604913604247, ""trx_score..."
3,2305465,9715ee51-7fac-4eda-b675-310d6ab79ac8,c7230c81-88fc-4337-a6cd-941c4b892b39,0.256490,2025-09-18T08:58:01.753671,2025-09-18T08:58:01.753671,Cash_beta_trench2_stackscore_backscore,v1,trench alpha beta,15620510-79ed-431f-b9ad-e196fd904819,2025-09-18T08:58:01.753671,{},"{""demo_score"": 0.34693482397153746, ""trx_score..."
4,2080167,567e6cf6-9f0f-4fb8-8ee5-1d0022d34d2a,9295238b-84fa-4ea7-882d-1d418d98802a,0.577553,2025-09-18T08:58:01.753671,2025-09-18T08:58:01.753671,Cash_beta_trench2_stackscore_backscore,v1,trench alpha beta,be095eaa-29eb-49dd-a309-315e180e6f81,2025-09-18T08:58:01.753671,{},"{""demo_score"": 0.6119191767689968, ""trx_score""..."


In [65]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [66]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench2_stackcore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_stackcore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_stackcore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_stackcore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench2_stackcore.joblib


# Insert into a table

In [67]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ecb5ca67-7045-4ba4-b7bd-450a9b67c125>

# cash_beta_trench3_applied_loans_backscored_20241001_20250831

# Table

In [68]:
schema1 = 'worktable_data_analysis'
cash_beta_trench3 = f'cash_beta_trench3_applied_loans_backscored_20241001_20250831'

# Query

In [69]:
sq = f"""
select * from {schema1}.{cash_beta_trench3};
"""
d1 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of {schema1}.{cash_beta_trench3} table is:\t {d1.shape}")

Job ID 4e6b1295-e59d-465a-b4d4-1a489bca36c0 successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of worktable_data_analysis.cash_beta_trench3_applied_loans_backscored_20241001_20250831 table is:	 (38281, 48)


In [70]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'c_t3_tx_cnt_installments_paid_tot_with_dpd',
       'c_t3_tx_time_since_last_applied_loan_application_time',
       'c_t3_tx_last_ap

In [71]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
           'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category', 'ln_loan_type', 'ln_disb_dtime'
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('demo_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench3_demoscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [72]:
transformeddata = f'cash_beta_trench3_democore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the cash_beta_trench3_democore data is: (38281, 13)


In [73]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,3478212,adbbb646-2cfd-45a0-8058-6eaa1c057234,3d00ed6e-c8c1-4e2f-9b5a-8a7f9698d5ab,0.399970,2025-09-18T08:58:53.795460,2025-09-18T08:58:53.795460,Cash_beta_trench3_demoscore_backscore,v1,trench alpha beta,742426bd-99b6-4dd7-b83b-18f3fbcd7c3d,2025-09-18T08:58:53.795460,{},"{""days_on_book"": 87, ""ln_appln_submit_datetime..."
1,2854066,3828ab3c-153f-456a-869a-c20c0bf665ac,9bcba59c-c33b-4df1-8b4e-51656eed52b6,0.485416,2025-09-18T08:58:53.795460,2025-09-18T08:58:53.795460,Cash_beta_trench3_demoscore_backscore,v1,trench alpha beta,5f7577f2-bb58-466b-9a79-8073f3ecb373,2025-09-18T08:58:53.795460,{},"{""days_on_book"": 14, ""ln_appln_submit_datetime..."
2,3193463,a1982acf-ec0f-4223-8aa8-a9cfe00d83cd,0b45c9bd-c9f7-4576-8548-f3173b86aa70,0.342219,2025-09-18T08:58:53.796460,2025-09-18T08:58:53.796460,Cash_beta_trench3_demoscore_backscore,v1,trench alpha beta,317e3c7c-7f6d-4230-987c-c5d17fc94327,2025-09-18T08:58:53.796460,{},"{""days_on_book"": 114, ""ln_appln_submit_datetim..."
3,3247272,9578e67e-54e2-405a-949b-484a72388652,25d21818-fc70-4ed4-845e-43575140efff,0.506910,2025-09-18T08:58:53.796460,2025-09-18T08:58:53.796460,Cash_beta_trench3_demoscore_backscore,v1,trench alpha beta,9eacac95-79c3-49f0-b47b-0d17c4246721,2025-09-18T08:58:53.796460,{},"{""days_on_book"": 39, ""ln_appln_submit_datetime..."
4,3084443,82cd0aba-977b-4dd5-8bbd-a4cde4c11736,b1f3d689-bcfb-494c-ae04-381676b773e4,0.349514,2025-09-18T08:58:53.796460,2025-09-18T08:58:53.796460,Cash_beta_trench3_demoscore_backscore,v1,trench alpha beta,14ec4017-a727-4ec7-b4d0-6acda4ea02e5,2025-09-18T08:58:53.796460,{},"{""days_on_book"": 129, ""ln_appln_submit_datetim..."


In [74]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [75]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench3_democore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_democore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_democore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_democore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_democore.joblib


# Insert into a table

In [76]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d103187d-582e-4096-bb9c-1232713e0616>

In [77]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'c_t3_tx_cnt_installments_paid_tot_with_dpd',
       'c_t3_tx_time_since_last_applied_loan_application_time',
       'c_t3_tx_last_ap

In [78]:
d2 = d1[d1['ln_os_type'] == 'Android'].copy()
d2['ln_os_type'].value_counts()

ln_os_type
Android    25678
Name: count, dtype: int64

In [79]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d2.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'ln_os_type', 'c_app_cnt_absence_tag_365d_binned',
        'c_app_cnt_books_and_reference_ever_binned',
        'c_app_cnt_gaming_180d_binned',
        'c_app_cnt_health_and_fitness_ever_binned',
        'c_app_cnt_productivity_ever_binned',
        'c_app_cnt_rated_for_18plus_ever_binned',
        'c_app_last_payday_install_to_apply_days_binned',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('apps_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench3_appscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [80]:
transformeddata = f'cash_beta_trench3_appcore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the cash_beta_trench3_appcore data is: (25678, 13)


In [81]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,1956508,30b44b43-6ef5-4d47-baeb-5168208b84af,8466178e-0c8f-4c05-ab5b-f8329cfb6bfe,0.379153,2025-09-18T08:59:17.675033,2025-09-18T08:59:17.675033,Cash_beta_trench3_appscore_backscore,v1,trench alpha beta,e15b2edf-124b-4f84-a3c6-e725c822e689,2025-09-18T08:59:17.675033,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_absence_t..."
1,2356633,d5e165f7-4e7d-4ed9-b8af-4da1d65bb642,8b3af73c-d6cd-48f2-8e9f-12b36945295c,0.380197,2025-09-18T08:59:17.676527,2025-09-18T08:59:17.676527,Cash_beta_trench3_appscore_backscore,v1,trench alpha beta,d8d272bc-312e-4435-96a9-1187433fa3fd,2025-09-18T08:59:17.676527,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_absence_t..."
2,3225371,1eb35d6a-7f60-4c3f-8171-c981e221b8e6,c1b2c5c0-cfc3-4329-8e3c-5c6d0a8527b1,0.391904,2025-09-18T08:59:17.676527,2025-09-18T08:59:17.676527,Cash_beta_trench3_appscore_backscore,v1,trench alpha beta,59faba6a-e013-4c3a-bd73-bed92fad736f,2025-09-18T08:59:17.676527,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_absence_t..."
3,2604750,069a097c-4afb-4c42-af3d-e61476eccfc9,05258b2a-9477-40aa-8008-a4b9bb9e8f2e,0.392407,2025-09-18T08:59:17.676527,2025-09-18T08:59:17.676527,Cash_beta_trench3_appscore_backscore,v1,trench alpha beta,c3cde570-53ea-4350-b487-e25a31d29cec,2025-09-18T08:59:17.676527,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_absence_t..."
4,2971573,2c2c6c99-8d5d-4709-9bc7-ba01ecc65676,28ff3e34-7860-4861-bb7e-37d686b66c5c,0.392909,2025-09-18T08:59:17.676527,2025-09-18T08:59:17.676527,Cash_beta_trench3_appscore_backscore,v1,trench alpha beta,ae7fb2b9-9c9f-48ec-b551-93436ddb0fb3,2025-09-18T08:59:17.676527,{},"{""ln_os_type"": ""Android"", ""c_app_cnt_absence_t..."


In [82]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [83]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench3_appcore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_appcore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_appcore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_appcore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_appcore.joblib


# Insert into a table

In [84]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=87654b99-4a28-43d9-bbb0-057638de4d09>

In [85]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'c_t3_tx_cnt_installments_paid_tot_with_dpd',
       'c_t3_tx_time_since_last_applied_loan_application_time',
       'c_t3_tx_last_ap

In [86]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'c_t3_tx_cnt_installments_paid_tot_with_dpd',
       'c_t3_tx_time_since_last_applied_loan_application_time',
       'c_t3_tx_last_applied_loan_decision', 'c_t3_tx_min_age_completed_loans',
       'c_t3_tx_dob_observation_date', 'c_t3_tx_cnt_jira_tickets_created_bin',
       'c_t3_tx_max_ever_dpd', 'c_t3_tx_amt_cash_in_total',
       'c_t3_tx_last_applied_loan_type_bin', 'c_t3_tx_cnt_completed_loans',
       'c_t3_tx_meng_no_of_logins', 'c_t3_tx_last_applied_loan_tenor',
       'c_t3_tx_med_days_bt_cash_out_trans',
       'c_t3_tx_avg_days_bt_cash_in_trans',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('trx_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench3_transactionscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [87]:
transformeddata = f'cash_beta_trench3_transactionscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the cash_beta_trench3_transactionscore data is: (38281, 13)


In [88]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,3478212,adbbb646-2cfd-45a0-8058-6eaa1c057234,baebc46d-e7c0-4a23-ad25-ff9928a1da98,0.587318,2025-09-18T08:59:33.311337,2025-09-18T08:59:33.311337,Cash_beta_trench3_transactionscore_backscore,v1,trench alpha beta,ace97638-3ca0-4a6a-b15b-510acad800c8,2025-09-18T08:59:33.311337,{},"{""c_t3_tx_cnt_installments_paid_tot_with_dpd"":..."
1,2854066,3828ab3c-153f-456a-869a-c20c0bf665ac,0976a3d7-866b-4672-b9dd-f58a81c28d0c,0.583157,2025-09-18T08:59:33.311337,2025-09-18T08:59:33.311337,Cash_beta_trench3_transactionscore_backscore,v1,trench alpha beta,c0ea05ec-0c43-4c8f-a08c-689af8824fa3,2025-09-18T08:59:33.311337,{},"{""c_t3_tx_cnt_installments_paid_tot_with_dpd"":..."
2,3193463,a1982acf-ec0f-4223-8aa8-a9cfe00d83cd,f0a6cff9-c3f8-487a-9a39-4e8cf7949698,0.607419,2025-09-18T08:59:33.312337,2025-09-18T08:59:33.312337,Cash_beta_trench3_transactionscore_backscore,v1,trench alpha beta,c3666c85-c640-4a27-bdbb-b3116d899b10,2025-09-18T08:59:33.312337,{},"{""c_t3_tx_cnt_installments_paid_tot_with_dpd"":..."
3,3247272,9578e67e-54e2-405a-949b-484a72388652,d70d8613-fc2f-4f06-922f-6c781c6107b6,0.586950,2025-09-18T08:59:33.312337,2025-09-18T08:59:33.312337,Cash_beta_trench3_transactionscore_backscore,v1,trench alpha beta,6d4e7e37-5cf0-4478-b2d6-6ebb8136d5b4,2025-09-18T08:59:33.312337,{},"{""c_t3_tx_cnt_installments_paid_tot_with_dpd"":..."
4,3084443,82cd0aba-977b-4dd5-8bbd-a4cde4c11736,f48a1707-544b-4b15-89b0-3bdd42a97e36,0.734227,2025-09-18T08:59:33.312337,2025-09-18T08:59:33.312337,Cash_beta_trench3_transactionscore_backscore,v1,trench alpha beta,411fac1c-3737-43f5-9792-523f6dc75bdc,2025-09-18T08:59:33.312337,{},"{""c_t3_tx_cnt_installments_paid_tot_with_dpd"":..."


In [89]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [90]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench3_transactionscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_transactionscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_transactionscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_transactionscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_transactionscore.joblib


# Insert into a table

In [91]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f492a0d7-bdbf-419a-9574-bcf49e4781cf>

In [92]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'demo_score',
       'trench_category', 'ln_loan_type', 'ln_disb_dtime',
       'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'c_t3_tx_cnt_installments_paid_tot_with_dpd',
       'c_t3_tx_time_since_last_applied_loan_application_time',
       'c_t3_tx_last_ap

In [93]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'demo_score', 'trx_score', 'apps_score', 'credo_score',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('stack_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_beta_trench3_stackscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [94]:
transformeddata = f'cash_beta_trench3_stackscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the cash_beta_trench3_stackscore data is: (38281, 13)


In [95]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,3478212,adbbb646-2cfd-45a0-8058-6eaa1c057234,cdcee396-f678-48a7-add8-4d3d903fcf7b,0.398694,2025-09-18T08:59:54.849478,2025-09-18T08:59:54.849478,Cash_beta_trench3_stackscore_backscore,v1,trench alpha beta,d0e1fa63-3422-4de8-a22e-3873fc8daa28,2025-09-18T08:59:54.849478,{},"{""demo_score"": 0.39996964766377463, ""trx_score..."
1,2854066,3828ab3c-153f-456a-869a-c20c0bf665ac,e6767389-8fce-4b49-841f-c73b086b321e,0.517808,2025-09-18T08:59:54.849478,2025-09-18T08:59:54.849478,Cash_beta_trench3_stackscore_backscore,v1,trench alpha beta,9154d26b-ce48-41aa-a535-d67988e63c0a,2025-09-18T08:59:54.849478,{},"{""demo_score"": 0.48541571842619596, ""trx_score..."
2,3193463,a1982acf-ec0f-4223-8aa8-a9cfe00d83cd,a07afea2-6ff5-465e-83dc-bc1cd6285d31,0.378178,2025-09-18T08:59:54.849478,2025-09-18T08:59:54.849478,Cash_beta_trench3_stackscore_backscore,v1,trench alpha beta,f3aff501-8150-4c2b-95b0-4b55934c5dff,2025-09-18T08:59:54.849478,{},"{""demo_score"": 0.34221888255492233, ""trx_score..."
3,3247272,9578e67e-54e2-405a-949b-484a72388652,22370a70-81fc-48b0-a0fb-66534db58097,0.523664,2025-09-18T08:59:54.849478,2025-09-18T08:59:54.849478,Cash_beta_trench3_stackscore_backscore,v1,trench alpha beta,ec91f4c7-1251-4c9e-821f-40bb9bc6d094,2025-09-18T08:59:54.849478,{},"{""demo_score"": 0.5069097879745724, ""trx_score""..."
4,3084443,82cd0aba-977b-4dd5-8bbd-a4cde4c11736,5f56a880-7668-4b77-8763-517aa14391e3,0.589999,2025-09-18T08:59:54.849478,2025-09-18T08:59:54.849478,Cash_beta_trench3_stackscore_backscore,v1,trench alpha beta,cb0c873f-1e1e-4b95-a07a-303fbd363689,2025-09-18T08:59:54.849478,{},"{""demo_score"": 0.34951373078873693, ""trx_score..."


In [96]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [97]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cash_beta_trench3_stackscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_stackscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_stackscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_stackscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cash_beta_trench3_stackscore.joblib


# Insert into a table

In [98]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=172f8c7d-9013-4bcf-a9d0-2a4ac92d0ca1>

# cash_alpha_trench1_applied_loans_backscored_20241001_20250831

# Table

In [99]:
schema1 = 'worktable_data_analysis'
cash_alpha_trench1 = f'cash_alpha_trench1_applied_loans_backscored_20241001_20250831'

# Query

In [100]:
sq = f"""
select * from {schema1}.{cash_alpha_trench1};
"""
d1 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of {schema1}.{cash_alpha_trench1} table is:\t {d1.shape}")

Job ID 91ed3b95-8872-4cd2-beca-349229427d2d successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of worktable_data_analysis.cash_alpha_trench1_applied_loans_backscored_20241001_20250831 table is:	 (62044, 48)


In [101]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_cic_max_age_all_contracts_snapshot',
       'ca_cic_ratio_overdue_contracts_to_granted_contracts',
       'ca_cic_ScoreRange', 'ca_cic_ln_loan_level_user_type',
       'ca_cic_has_ever_been_overdue',
  

In [102]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
         'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('demo_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench1_demoscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [103]:
transformeddata = f'Cash_alpha_trench1_demoscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench1_demoscore_backscore data is: (62044, 13)


In [104]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2945653,e7e3e1dc-de0e-4ad8-b374-2cd2ab2bd87a,9488a185-a954-48a4-bc49-9f804ddb20c3,0.504412,2025-09-18T09:00:47.357589,2025-09-18T09:00:47.357589,Cash_alpha_trench1_demoscore_backscore,v1,trench alpha beta,02f41df0-8937-47cc-90a9-2bdc9a438f16,2025-09-18T09:00:47.357589,{},"{""days_on_book"": 0, ""ln_appln_submit_datetime""..."
1,3436627,e07c1df0-56cd-4d5f-bfd4-8deb3afdc95e,80d86984-5f43-4445-b0be-e1a3e9856d29,0.447430,2025-09-18T09:00:47.357589,2025-09-18T09:00:47.357589,Cash_alpha_trench1_demoscore_backscore,v1,trench alpha beta,d8e48ece-b7b0-4392-a02e-e1ecd86e0a2a,2025-09-18T09:00:47.357589,{},"{""days_on_book"": 0, ""ln_appln_submit_datetime""..."
2,3110665,bac4a185-04bf-4c90-b73e-2ee0d6707df4,059fe90f-b957-4805-8679-48da2ecaf218,0.453835,2025-09-18T09:00:47.357589,2025-09-18T09:00:47.357589,Cash_alpha_trench1_demoscore_backscore,v1,trench alpha beta,6803ef49-b0d2-47d0-bff3-43de1c37a5b8,2025-09-18T09:00:47.357589,{},"{""days_on_book"": 0, ""ln_appln_submit_datetime""..."
3,3143585,cc03f3f1-4132-4ed7-9e14-8a096139acc6,f64b796c-159e-4b6f-9e8c-00b886aecd82,0.395059,2025-09-18T09:00:47.357589,2025-09-18T09:00:47.357589,Cash_alpha_trench1_demoscore_backscore,v1,trench alpha beta,8115455b-8ee0-4a98-9bda-260863c0efb1,2025-09-18T09:00:47.357589,{},"{""days_on_book"": 0, ""ln_appln_submit_datetime""..."
4,3616924,f3bd7ba1-a1a9-4868-97ab-ad1a75799f60,caa46ee5-06d3-4ba1-b7fc-7623233dd5c1,0.570916,2025-09-18T09:00:47.358589,2025-09-18T09:00:47.358589,Cash_alpha_trench1_demoscore_backscore,v1,trench alpha beta,cdffe633-5257-4ffd-b4b3-0f37e25cbda5,2025-09-18T09:00:47.358589,{},"{""days_on_book"": 6, ""ln_appln_submit_datetime""..."


In [105]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [106]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench1_demoscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_demoscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_demoscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_demoscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_demoscore_backscore.joblib


# Insert into a table

In [107]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=632f971b-89d0-4987-afdb-b64e0d4f8874>

In [108]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_cic_max_age_all_contracts_snapshot',
       'ca_cic_ratio_overdue_contracts_to_granted_contracts',
       'ca_cic_ScoreRange', 'ca_cic_ln_loan_level_user_type',
       'ca_cic_has_ever_been_overdue',
  

In [109]:
d2 = d1[d1['ln_os_type'] == 'Android']
d2['ln_os_type'].value_counts()

ln_os_type
Android    28364
Name: count, dtype: int64

In [110]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d2.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
         'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('apps_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench1_appscorescore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [111]:
transformeddata = f'Cash_alpha_trench1_appscorescore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench1_appscorescore_backscore data is: (28364, 13)


In [112]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,3411924,a21d0c56-2495-4bf1-a529-c2344d5583b0,f88b7aaa-0361-4b4c-b074-18c97e6545a9,0.679013,2025-09-18T09:01:23.391205,2025-09-18T09:01:23.391205,Cash_alpha_trench1_appscorescore_backscore,v1,trench alpha beta,7c7e2fc2-f9ee-4866-bfb0-e561b1b79839,2025-09-18T09:01:23.391205,{},"{""ca_app_cnt_health_and_fitness_ever"": 0.0, ""c..."
1,3493655,0b5b65b9-45c3-41b9-8851-57b3a0565450,20e96660-6a2f-43d2-a782-3b9baa7034f2,0.660629,2025-09-18T09:01:23.391205,2025-09-18T09:01:23.391205,Cash_alpha_trench1_appscorescore_backscore,v1,trench alpha beta,bff22ab2-8aba-4f44-9d0d-31fe6aabaf1e,2025-09-18T09:01:23.391205,{},"{""ca_app_cnt_health_and_fitness_ever"": 0.0, ""c..."
2,3199802,8b7801e5-56dd-4dc1-bcf3-9730aab8c9fc,6a0d159d-e051-4694-861c-f49c9645d1cf,0.495682,2025-09-18T09:01:23.392205,2025-09-18T09:01:23.392205,Cash_alpha_trench1_appscorescore_backscore,v1,trench alpha beta,e2190669-2ea7-4ac6-89ef-3fde8d31cf3a,2025-09-18T09:01:23.392205,{},"{""ca_app_cnt_health_and_fitness_ever"": 0.0, ""c..."
3,3402738,43e5b020-70ec-4839-abe6-2bbd49ac866b,00955d18-2279-4c90-8eee-9055a4e87271,0.690475,2025-09-18T09:01:23.392205,2025-09-18T09:01:23.392205,Cash_alpha_trench1_appscorescore_backscore,v1,trench alpha beta,9e6b826f-98b2-490f-825b-10e9b0ae763a,2025-09-18T09:01:23.392205,{},"{""ca_app_cnt_health_and_fitness_ever"": 0.0, ""c..."
4,3474016,2d426e39-2eed-4485-8e8d-17b373f5f950,53665f21-189a-40f5-953e-55c27a1eece2,0.708802,2025-09-18T09:01:23.392205,2025-09-18T09:01:23.392205,Cash_alpha_trench1_appscorescore_backscore,v1,trench alpha beta,2109a09f-59b9-487c-8b85-a39be0eb7ff8,2025-09-18T09:01:23.392205,{},"{""ca_app_cnt_health_and_fitness_ever"": 0.0, ""c..."


In [113]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [114]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench1_appscorescore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_appscorescore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_appscorescore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_appscorescore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_appscorescore_backscore.joblib


# Insert into a table

In [115]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8e62eb28-e01d-4618-b380-e1717c27f3b5>

In [116]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_cic_max_age_all_contracts_snapshot',
       'ca_cic_ratio_overdue_contracts_to_granted_contracts',
       'ca_cic_ScoreRange', 'ca_cic_ln_loan_level_user_type',
       'ca_cic_has_ever_been_overdue',
  

In [117]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
       'ca_cic_max_age_all_contracts_snapshot',
       'ca_cic_ratio_overdue_contracts_to_granted_contracts',
       'ca_cic_ScoreRange', 'ca_cic_ln_loan_level_user_type',
       'ca_cic_has_ever_been_overdue',
       'ca_cic_latest_granted_contract_overdue_flag',
       'ca_cic_ratio_closed_over_new_granted_cnt_24M',
       'ca_cic_ratio_risky_contracts_to_granted_contracts',
       'ca_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'ca_cic_flg_zero_non_granted_ever',
       'ca_cic_Personal_Loans_granted_contracts_amt_24M',
       'ca_cic_CreditAvgCreditLimit', 'ca_cic_flg_zero_granted_ever',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('cic_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench1_cicscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [118]:
transformeddata = f'Cash_alpha_trench1_cicscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench1_cicscore_backscore data is: (62044, 13)


In [119]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2945653,e7e3e1dc-de0e-4ad8-b374-2cd2ab2bd87a,a33054c5-2998-43e5-94de-b1b53da8c61d,0.568481,2025-09-18T09:01:37.867822,2025-09-18T09:01:37.867822,Cash_alpha_trench1_cicscore_backscore,v1,trench alpha beta,3c06f187-bb8c-4ac6-87d7-3fe5da7ef339,2025-09-18T09:01:37.867822,{},"{""ca_cic_max_age_all_contracts_snapshot"": 357...."
1,3436627,e07c1df0-56cd-4d5f-bfd4-8deb3afdc95e,491d5fe1-62f0-426e-a795-4bdc18e41e70,0.711520,2025-09-18T09:01:37.867822,2025-09-18T09:01:37.867822,Cash_alpha_trench1_cicscore_backscore,v1,trench alpha beta,94357455-451c-4d9f-b6be-434a9fb1ec84,2025-09-18T09:01:37.867822,{},"{""ca_cic_max_age_all_contracts_snapshot"": 133...."
2,3110665,bac4a185-04bf-4c90-b73e-2ee0d6707df4,e79c2f3f-7537-441e-8299-5a9538090a69,0.583933,2025-09-18T09:01:37.868820,2025-09-18T09:01:37.868820,Cash_alpha_trench1_cicscore_backscore,v1,trench alpha beta,982cc708-9258-4e56-b422-4a852decb2f6,2025-09-18T09:01:37.868820,{},"{""ca_cic_max_age_all_contracts_snapshot"": 1740..."
3,3143585,cc03f3f1-4132-4ed7-9e14-8a096139acc6,981d2741-79d6-4b4d-b829-98d56f061a45,0.409816,2025-09-18T09:01:37.869236,2025-09-18T09:01:37.869236,Cash_alpha_trench1_cicscore_backscore,v1,trench alpha beta,4447e5fc-1967-47e3-b64a-27cf264f04d3,2025-09-18T09:01:37.869236,{},"{""ca_cic_max_age_all_contracts_snapshot"": 3493..."
4,3616924,f3bd7ba1-a1a9-4868-97ab-ad1a75799f60,5365fcf8-c54c-491e-8670-ed519c7985df,0.507959,2025-09-18T09:01:37.869236,2025-09-18T09:01:37.869236,Cash_alpha_trench1_cicscore_backscore,v1,trench alpha beta,04299eae-57c1-43d6-b968-fe680c48a7c5,2025-09-18T09:01:37.869236,{},"{""ca_cic_max_age_all_contracts_snapshot"": 1688..."


In [120]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [121]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench1_cicscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_cicscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_cicscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_cicscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_cicscore_backscore.joblib


# Insert into a table

In [122]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e72a25b1-614a-49bb-890b-4005eae2f933>

In [123]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_cic_max_age_all_contracts_snapshot',
       'ca_cic_ratio_overdue_contracts_to_granted_contracts',
       'ca_cic_ScoreRange', 'ca_cic_ln_loan_level_user_type',
       'ca_cic_has_ever_been_overdue',
  

In [124]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'demo_score', 'apps_score', 'credo_score','cic_score',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('stack_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench1_stackscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [125]:
transformeddata = f'Cash_alpha_trench1_stackscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench1_stackscore_backscore data is: (62044, 13)


In [126]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2945653,e7e3e1dc-de0e-4ad8-b374-2cd2ab2bd87a,4d9490de-d803-4704-a053-4d54ad041178,0.620850,2025-09-18T09:02:10.306514,2025-09-18T09:02:10.306514,Cash_alpha_trench1_stackscore_backscore,v1,trench alpha beta,b3861860-e5d0-41ac-98c8-ad1dd154efc1,2025-09-18T09:02:10.306514,{},"{""demo_score"": 0.504411926197625, ""credo_score..."
1,3436627,e07c1df0-56cd-4d5f-bfd4-8deb3afdc95e,c9c027ff-449d-4af6-bde8-47f98bdc9946,0.640329,2025-09-18T09:02:10.306514,2025-09-18T09:02:10.306514,Cash_alpha_trench1_stackscore_backscore,v1,trench alpha beta,2e8e3db7-a104-479c-8e4f-03bd8e08fd67,2025-09-18T09:02:10.306514,{},"{""demo_score"": 0.44742975606632246, ""credo_sco..."
2,3110665,bac4a185-04bf-4c90-b73e-2ee0d6707df4,cb690d89-993e-49df-935b-42d9da7d0441,0.534009,2025-09-18T09:02:10.306514,2025-09-18T09:02:10.306514,Cash_alpha_trench1_stackscore_backscore,v1,trench alpha beta,d788fc3e-2782-48b4-bec8-cc0dcb3d8468,2025-09-18T09:02:10.306514,{},"{""demo_score"": 0.45383506885111236, ""credo_sco..."
3,3143585,cc03f3f1-4132-4ed7-9e14-8a096139acc6,5fc85c1e-a34d-4d41-8b39-ae2c57d671e1,0.243430,2025-09-18T09:02:10.306514,2025-09-18T09:02:10.306514,Cash_alpha_trench1_stackscore_backscore,v1,trench alpha beta,dd553d6d-6e42-4068-8459-7c5fae97329c,2025-09-18T09:02:10.306514,{},"{""demo_score"": 0.39505892294939876, ""credo_sco..."
4,3616924,f3bd7ba1-a1a9-4868-97ab-ad1a75799f60,b6e1f159-44be-4002-9eb2-e24d66a0a72e,0.553493,2025-09-18T09:02:10.306514,2025-09-18T09:02:10.306514,Cash_alpha_trench1_stackscore_backscore,v1,trench alpha beta,f7190ebe-2d86-4572-bee0-c1f69418fcb8,2025-09-18T09:02:10.306514,{},"{""demo_score"": 0.5709163633237098, ""credo_scor..."


In [127]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [128]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench1_stackscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_stackscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_stackscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_stackscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench1_stackscore_backscore.joblib


# Insert into a table

In [129]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=86ad560b-84b3-4e41-85b7-6fa598108c34>

# cash_alpha_trench2_applied_loans_backscored_20241001_20250831

# Table

In [130]:
schema1 = 'worktable_data_analysis'
cash_alpha_trench2 = f'cash_alpha_trench2_applied_loans_backscored_20241001_20250831'

# Query

In [131]:
sq = f"""
select * from {schema1}.{cash_alpha_trench2};
"""
d1 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of {schema1}.{cash_alpha_trench2} table is:\t {d1.shape}")

Job ID 76cf8f6e-1370-4274-ac54-440691c82fcb successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of worktable_data_analysis.cash_alpha_trench2_applied_loans_backscored_20241001_20250831 table is:	 (39651, 60)


In [132]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'ca_t2_tx_first_product_user_segment_WOE',
       'ca_t2_tx_first_applied_loan_type_bin_WOE',
       'ca_t2_tx_cnt_rejected_loans',
       'ca_t2_tx_appsf

In [133]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
         'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
        'trench_category',  'ln_loan_type', 'ln_disb_dtime',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('demo_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench2_demoscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [134]:
transformeddata = f'Cash_alpha_trench2_demoscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench2_demoscore_backscore data is: (39651, 13)


In [135]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2278578,527a7986-f598-4498-8dbe-9f5baa0397ce,2b4fbea0-0ec5-4fbe-bad6-533cd8d131f5,0.591407,2025-09-18T09:03:04.513349,2025-09-18T09:03:04.513349,Cash_alpha_trench2_demoscore_backscore,v1,trench alpha beta,2086d42b-69ef-4ed5-9812-a695e48a40dd,2025-09-18T09:03:04.513349,{},"{""days_on_book"": 421, ""ln_appln_submit_datetim..."
1,2808330,18122fb1-947f-4004-9033-c4bdcbbca5e6,46661232-3e59-491c-9540-a2629cfd417d,0.464558,2025-09-18T09:03:04.514349,2025-09-18T09:03:04.514349,Cash_alpha_trench2_demoscore_backscore,v1,trench alpha beta,7ae47ef1-e2c6-4736-b43a-bcde0ea85607,2025-09-18T09:03:04.514349,{},"{""days_on_book"": 40, ""ln_appln_submit_datetime..."
2,3403996,a087c785-8d47-49d7-83f1-896dfc0b007a,12ab4a31-3a46-4fde-a9bc-4712d2d84190,0.460365,2025-09-18T09:03:04.514349,2025-09-18T09:03:04.514349,Cash_alpha_trench2_demoscore_backscore,v1,trench alpha beta,aed136c3-f95c-4767-a754-def7dcd192cc,2025-09-18T09:03:04.514349,{},"{""days_on_book"": 415, ""ln_appln_submit_datetim..."
3,3172423,965de26f-3ab6-4992-b678-f7f6f8219736,caa96974-db13-4724-8694-e1d0be3ee137,0.583504,2025-09-18T09:03:04.515349,2025-09-18T09:03:04.515349,Cash_alpha_trench2_demoscore_backscore,v1,trench alpha beta,643e1ff5-8f06-41d1-88ff-6d2fe3597a6e,2025-09-18T09:03:04.515349,{},"{""days_on_book"": 31, ""ln_appln_submit_datetime..."
4,2422495,59483165-7e09-4ab2-a038-c13577f02479,48a57e84-d8ba-4df0-9d2a-60109d46bfb0,0.433422,2025-09-18T09:03:04.515349,2025-09-18T09:03:04.515349,Cash_alpha_trench2_demoscore_backscore,v1,trench alpha beta,42510e11-91a8-4f2e-8025-586dd3681b9f,2025-09-18T09:03:04.515349,{},"{""days_on_book"": 241, ""ln_appln_submit_datetim..."


In [136]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [137]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench2_demoscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_demoscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_demoscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_demoscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_demoscore_backscore.joblib


# Insert into a table

In [138]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6f6a7718-ce11-4b4b-ad4f-aa87b09a65a0>

# Trench2 app score

In [139]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'ca_t2_tx_first_product_user_segment_WOE',
       'ca_t2_tx_first_applied_loan_type_bin_WOE',
       'ca_t2_tx_cnt_rejected_loans',
       'ca_t2_tx_appsf

In [140]:
d2 = d1[d1['ln_os_type'] == 'Android'].copy()
d2['ln_os_type'].value_counts()

ln_os_type
Android    22572
Name: count, dtype: int64

In [141]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d2.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
          'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('apps_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench2_appscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [142]:
transformeddata = f'Cash_alpha_trench2_appscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench2_appscore_backscore data is: (22572, 13)


In [143]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2899910,7396933c-2a9d-44f5-b733-7a2729d069cc,b3020622-9a85-4398-9852-6890c3cd789b,0.478660,2025-09-18T09:03:30.820313,2025-09-18T09:03:30.820313,Cash_alpha_trench2_appscore_backscore,v1,trench alpha beta,f525d4f5-ece0-4d45-aada-a79595654a1c,2025-09-18T09:03:30.820313,{},"{""ca_app_cnt_health_and_fitness_ever"": 0.0, ""c..."
1,1837691,417f4154-8bdf-41fc-8599-9da2a02b3ae0,99c02348-c44f-4703-b0cf-e44af1315c1a,0.487730,2025-09-18T09:03:30.820313,2025-09-18T09:03:30.820313,Cash_alpha_trench2_appscore_backscore,v1,trench alpha beta,bdd15765-603a-4f53-a7a6-6482e16093dd,2025-09-18T09:03:30.820313,{},"{""ca_app_cnt_health_and_fitness_ever"": 4.0, ""c..."
2,1003738,6c0b5dda-1b78-4960-a912-58be3c0a23a9,c353ce4c-741f-428d-8a62-08a5deeed93b,0.377221,2025-09-18T09:03:30.821313,2025-09-18T09:03:30.821313,Cash_alpha_trench2_appscore_backscore,v1,trench alpha beta,8d21960c-dcdf-406e-92ae-a803b4f0b3fd,2025-09-18T09:03:30.821313,{},"{""ca_app_cnt_health_and_fitness_ever"": 1.0, ""c..."
3,1762559,5a37b417-d7fc-482e-8a54-0743b93293b8,e5ceb67d-0bb5-4179-b58c-5324a031c535,0.278161,2025-09-18T09:03:30.821313,2025-09-18T09:03:30.821313,Cash_alpha_trench2_appscore_backscore,v1,trench alpha beta,16c8961d-c69d-4a88-b1ef-5e9764758214,2025-09-18T09:03:30.821313,{},"{""ca_app_cnt_health_and_fitness_ever"": 2.0, ""c..."
4,3074855,30e08b8f-93fd-47a2-b14b-028292e4d93a,7db74fb2-256a-4314-9a4d-4e036d42a3ed,0.637516,2025-09-18T09:03:30.821313,2025-09-18T09:03:30.821313,Cash_alpha_trench2_appscore_backscore,v1,trench alpha beta,c3e1b13a-611f-4da4-90ab-9e6899e367f4,2025-09-18T09:03:30.821313,{},"{""ca_app_cnt_health_and_fitness_ever"": 0.0, ""c..."


In [144]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [145]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench2_appscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_appscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_appscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_appscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_appscore_backscore.joblib


# Insert into a table

In [146]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=456e99b2-949b-493d-a27e-804fe9e399e4>

# Trench2 transaction score

In [147]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'ca_t2_tx_first_product_user_segment_WOE',
       'ca_t2_tx_first_applied_loan_type_bin_WOE',
       'ca_t2_tx_cnt_rejected_loans',
       'ca_t2_tx_appsf

In [148]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
           'ca_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'ca_t2_tx_first_product_user_segment_WOE',
       'ca_t2_tx_first_applied_loan_type_bin_WOE',
       'ca_t2_tx_cnt_rejected_loans',
       'ca_t2_tx_appsflyer_install_to_registration_minutes',
       'ca_t2_tx_first_applied_loan_amount', 'ca_t2_tx_deposit_accnt_cnt',
       'ca_t2_tx_cnt_cash_in_total', 'ca_t2_tx_cnt_incomplete_loan_apps',
       'ca_t2_tx_amt_cash_in_total',
       'ca_t2_tx_last_applied_loan_tenor_bin_WOE', 
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('trx_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench2_transactionscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [149]:
transformeddata = f'trx_score'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the trx_score data is: (39651, 13)


In [150]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2278578,527a7986-f598-4498-8dbe-9f5baa0397ce,ed871a80-0326-4328-b36b-3e937dd7c132,0.544192,2025-09-18T09:03:48.792147,2025-09-18T09:03:48.792147,Cash_alpha_trench2_transactionscore_backscore,v1,trench alpha beta,3c756075-f06d-4c1d-9a94-49cb30822921,2025-09-18T09:03:48.792147,{},"{""ca_t2_tx_meng_ql_calculator_tot_visit_cnt"": ..."
1,2808330,18122fb1-947f-4004-9033-c4bdcbbca5e6,5f761217-abe5-4818-99b8-44652e3d4e33,0.147100,2025-09-18T09:03:48.793150,2025-09-18T09:03:48.793150,Cash_alpha_trench2_transactionscore_backscore,v1,trench alpha beta,9f9ef41e-97c9-4d33-a74b-243188b36054,2025-09-18T09:03:48.793150,{},"{""ca_t2_tx_meng_ql_calculator_tot_visit_cnt"": ..."
2,3403996,a087c785-8d47-49d7-83f1-896dfc0b007a,dcc9155b-77cf-49b9-8092-226fd7f47b77,0.463880,2025-09-18T09:03:48.793150,2025-09-18T09:03:48.793150,Cash_alpha_trench2_transactionscore_backscore,v1,trench alpha beta,5b09172a-482a-4b43-8bc0-480dab3a5e93,2025-09-18T09:03:48.793150,{},"{""ca_t2_tx_meng_ql_calculator_tot_visit_cnt"": ..."
3,3172423,965de26f-3ab6-4992-b678-f7f6f8219736,0f26225a-0c48-49a4-bcdc-e39aaa0aa159,0.509618,2025-09-18T09:03:48.793150,2025-09-18T09:03:48.793150,Cash_alpha_trench2_transactionscore_backscore,v1,trench alpha beta,0872710f-e603-41d6-a106-ee3593be90e9,2025-09-18T09:03:48.793150,{},"{""ca_t2_tx_meng_ql_calculator_tot_visit_cnt"": ..."
4,2422495,59483165-7e09-4ab2-a038-c13577f02479,5eed6821-ce68-474a-8c9e-6bbedb472deb,0.432925,2025-09-18T09:03:48.794148,2025-09-18T09:03:48.794148,Cash_alpha_trench2_transactionscore_backscore,v1,trench alpha beta,d95ff888-d844-4d85-8d6c-40376c6c1f29,2025-09-18T09:03:48.794148,{},"{""ca_t2_tx_meng_ql_calculator_tot_visit_cnt"": ..."


In [151]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [152]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_trx_score
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_trx_score.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_trx_score.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_trx_score.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_trx_score.joblib


# Insert into a table

In [153]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2aeefc9d-8331-429e-8dc1-88f4b4b31d08>

# Alpha trench2 cic score

In [154]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'ca_t2_tx_first_product_user_segment_WOE',
       'ca_t2_tx_first_applied_loan_type_bin_WOE',
       'ca_t2_tx_cnt_rejected_loans',
       'ca_t2_tx_appsf

In [155]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
           'ca_cic_max_age_all_contracts_snapshot',
       'ca_cic_ratio_overdue_contracts_to_granted_contracts',
       'ca_cic_ScoreRange', 'ca_cic_ln_loan_level_user_type',
       'ca_cic_has_ever_been_overdue',
       'ca_cic_latest_granted_contract_overdue_flag',
       'ca_cic_ratio_closed_over_new_granted_cnt_24M',
       'ca_cic_ratio_risky_contracts_to_granted_contracts',
       'ca_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'ca_cic_flg_zero_non_granted_ever',
       'ca_cic_Personal_Loans_granted_contracts_amt_24M',
       'ca_cic_CreditAvgCreditLimit', 'ca_cic_flg_zero_granted_ever',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('cic_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench2_cicscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [156]:
transformeddata = f'cic_score'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the cic_score data is: (39651, 13)


In [157]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2278578,527a7986-f598-4498-8dbe-9f5baa0397ce,fa7c6d85-c019-4db1-8f77-318b316597e3,0.748580,2025-09-18T09:04:11.396678,2025-09-18T09:04:11.396678,Cash_alpha_trench2_cicscore_backscore,v1,trench alpha beta,09e82850-7f05-4f15-9c10-5a5ff79f4d8f,2025-09-18T09:04:11.396678,{},"{""ca_cic_max_age_all_contracts_snapshot"": 730...."
1,2808330,18122fb1-947f-4004-9033-c4bdcbbca5e6,287453ec-a662-4203-a1b1-3e8f8d1d6def,0.413837,2025-09-18T09:04:11.396678,2025-09-18T09:04:11.396678,Cash_alpha_trench2_cicscore_backscore,v1,trench alpha beta,f98d1ba9-7514-4552-9d94-a75747a82705,2025-09-18T09:04:11.396678,{},"{""ca_cic_max_age_all_contracts_snapshot"": 2646..."
2,3403996,a087c785-8d47-49d7-83f1-896dfc0b007a,20bf3d48-abc8-427a-b1ee-fcd1068ef2f6,0.506584,2025-09-18T09:04:11.397679,2025-09-18T09:04:11.397679,Cash_alpha_trench2_cicscore_backscore,v1,trench alpha beta,5aa1d5d8-cdd8-486a-b4b5-0a92570439c1,2025-09-18T09:04:11.397679,{},"{""ca_cic_max_age_all_contracts_snapshot"": 89.0..."
3,3172423,965de26f-3ab6-4992-b678-f7f6f8219736,ca738a66-8c17-4a35-ae79-d52b6ea87712,0.487934,2025-09-18T09:04:11.397679,2025-09-18T09:04:11.397679,Cash_alpha_trench2_cicscore_backscore,v1,trench alpha beta,3d9485ee-f684-47eb-b4f3-29a544ce2f59,2025-09-18T09:04:11.397679,{},"{""ca_cic_max_age_all_contracts_snapshot"": 471...."
4,2422495,59483165-7e09-4ab2-a038-c13577f02479,f79cb468-f0f7-459a-8b97-4d727c814f21,0.394879,2025-09-18T09:04:11.397679,2025-09-18T09:04:11.397679,Cash_alpha_trench2_cicscore_backscore,v1,trench alpha beta,4eb4b649-b1b2-4a87-9528-99c0cc7a357b,2025-09-18T09:04:11.397679,{},"{""ca_cic_max_age_all_contracts_snapshot"": 2085..."


In [158]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [159]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_cic_score
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cic_score.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cic_score.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cic_score.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_cic_score.joblib


# Insert into a table

In [160]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=89ecd7fc-04e2-4dc8-9912-bae791a3cd2d>

# Alpha trench2 Stack Score

In [161]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'ca_app_cnt_health_and_fitness_ever',
       'ca_app_cnt_shopping_ever', 'ca_app_median_time_bw_installed_mins_ever',
       'ca_app_avg_time_bw_installed_mins_3d', 'ca_app_cnt_crypto_ever',
       'ca_app_cnt_driver_ever', 'ca_app_cnt_payday_180d',
       'ca_app_cnt_gambling_180d', 'ca_t2_tx_meng_ql_calculator_tot_visit_cnt',
       'ca_t2_tx_first_product_user_segment_WOE',
       'ca_t2_tx_first_applied_loan_type_bin_WOE',
       'ca_t2_tx_cnt_rejected_loans',
       'ca_t2_tx_appsf

In [162]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
           'demo_score', 'apps_score', 'credo_score', 'cic_score',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('stack_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench2_stackscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [163]:
transformeddata = f'Cash_alpha_trench2_stackscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench2_stackscore_backscore data is: (39651, 13)


In [164]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,2278578,527a7986-f598-4498-8dbe-9f5baa0397ce,d4ef8fe8-c150-4cbd-8e1c-f6cb8c570979,0.867653,2025-09-18T09:04:31.852816,2025-09-18T09:04:31.852816,Cash_alpha_trench2_stackscore_backscore,v1,trench alpha beta,d881d452-a78e-4826-b97e-c4f80d2c3e98,2025-09-18T09:04:31.852816,{},"{""demo_score"": 0.5914070310856829, ""credo_scor..."
1,2808330,18122fb1-947f-4004-9033-c4bdcbbca5e6,5e218973-da91-486a-84e4-3e2e59cb123a,0.203759,2025-09-18T09:04:31.852816,2025-09-18T09:04:31.852816,Cash_alpha_trench2_stackscore_backscore,v1,trench alpha beta,62ce2342-3348-4d95-8abe-3fd33ed39b8b,2025-09-18T09:04:31.852816,{},"{""demo_score"": 0.4645575141164843, ""credo_scor..."
2,3403996,a087c785-8d47-49d7-83f1-896dfc0b007a,9d96066d-3182-4b33-a8ae-2c2bc76dc6f9,0.370856,2025-09-18T09:04:31.852816,2025-09-18T09:04:31.852816,Cash_alpha_trench2_stackscore_backscore,v1,trench alpha beta,091db2bf-a1c2-4ffd-90b7-07ddb1aa4ed1,2025-09-18T09:04:31.852816,{},"{""demo_score"": 0.4603653023836028, ""credo_scor..."
3,3172423,965de26f-3ab6-4992-b678-f7f6f8219736,a014e5f8-d643-4cf7-a97e-75463eaa4aad,0.540000,2025-09-18T09:04:31.852816,2025-09-18T09:04:31.852816,Cash_alpha_trench2_stackscore_backscore,v1,trench alpha beta,954bd8fc-51e7-4dbf-90fd-7cf027840e50,2025-09-18T09:04:31.852816,{},"{""demo_score"": 0.5835036923589388, ""credo_scor..."
4,2422495,59483165-7e09-4ab2-a038-c13577f02479,45261c9a-01f6-4456-acb8-1add11afb4b8,0.228603,2025-09-18T09:04:31.852816,2025-09-18T09:04:31.852816,Cash_alpha_trench2_stackscore_backscore,v1,trench alpha beta,d8f84ff0-165a-44dd-bb94-91af5ccf9aef,2025-09-18T09:04:31.852816,{},"{""demo_score"": 0.43342198257276704, ""credo_sco..."


In [165]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [166]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench2_stackscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_stackscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_stackscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_stackscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench2_stackscore_backscore.joblib


# Insert into a table

In [167]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=164c96ee-ce08-4c25-b552-c764dd1bc10a>

# cash_alpha_trench3_applied_loans_backscored_20241001_20250831

# Table

In [168]:
schema1 = 'worktable_data_analysis'
cash_alpha_trench3 = f'cash_alpha_trench3_applied_loans_backscored_20241001_20250831'

# Query

In [169]:
sq = f"""
select * from {schema1}.{cash_alpha_trench3};
"""
d1 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of {schema1}.{cash_alpha_trench3} table is:\t {d1.shape}")

Job ID a728e485-bb85-4d9d-8b7f-1f7636f7e542 successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of worktable_data_analysis.cash_alpha_trench3_applied_loans_backscored_20241001_20250831 table is:	 (16504, 62)


# Alpha trench3 Demo Score

In [170]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'ca_t3_tx_cnt_installments_paid_tot_with_dpd',
       'ca_t3_tx_time_since_last_applied_loan_application_time',
       'ca_t3_tx_last_applied_loan_decisio

In [171]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
           'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'trench_category',  'ln_loan_type', 'ln_disb_dtime',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('demo_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench3_demoscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [172]:
transformeddata = f'Cash_alpha_trench3_demoscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench3_demoscore_backscore data is: (16504, 13)


In [173]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,1854550,7aa29106-8b0c-4749-ba35-2bc56713b481,44d5de91-4b83-4308-a2a5-04fa50ce51fc,0.416857,2025-09-18T09:05:00.876488,2025-09-18T09:05:00.876488,Cash_alpha_trench3_demoscore_backscore,v1,trench alpha beta,fd797e2a-7054-48b0-af29-b1d6313dc8d3,2025-09-18T09:05:00.876488,{},"{""days_on_book"": 1028, ""ln_appln_submit_dateti..."
1,2270274,f3157ede-9e52-409d-a360-a0b8e4a7df9a,511cb71b-7355-4a94-8ed4-834fe6674d4f,0.526911,2025-09-18T09:05:00.877490,2025-09-18T09:05:00.877490,Cash_alpha_trench3_demoscore_backscore,v1,trench alpha beta,371051ee-4e64-487e-867c-4074376b82e4,2025-09-18T09:05:00.877490,{},"{""days_on_book"": 358, ""ln_appln_submit_datetim..."
2,1590642,c70c826c-78d6-4dcd-ac11-aae802e26645,32f263d6-6345-49ff-80b3-119165b5f51d,0.512391,2025-09-18T09:05:00.878488,2025-09-18T09:05:00.878488,Cash_alpha_trench3_demoscore_backscore,v1,trench alpha beta,0501becc-be9f-4f6e-9fe9-4859ff906be6,2025-09-18T09:05:00.878488,{},"{""days_on_book"": 887, ""ln_appln_submit_datetim..."
3,2421648,e59cb3e5-0f78-4c81-bfc7-e3d3e28a5dfb,cba174d3-739b-48da-ad9b-99619f784759,0.423516,2025-09-18T09:05:00.878488,2025-09-18T09:05:00.878488,Cash_alpha_trench3_demoscore_backscore,v1,trench alpha beta,8f534810-6541-40dc-b7d5-d5c4639cffe0,2025-09-18T09:05:00.878488,{},"{""days_on_book"": 309, ""ln_appln_submit_datetim..."
4,1411743,fd9be7a5-f0c9-4110-a45a-fcfe9736d665,5415f815-8138-4ff0-8cf1-6cab4b84229d,0.518323,2025-09-18T09:05:00.879065,2025-09-18T09:05:00.879065,Cash_alpha_trench3_demoscore_backscore,v1,trench alpha beta,6e545e7e-4704-4acd-82ae-beb81492bcf1,2025-09-18T09:05:00.879065,{},"{""days_on_book"": 847, ""ln_appln_submit_datetim..."


In [174]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [175]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench3_demoscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_demoscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_demoscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_demoscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_demoscore_backscore.joblib


# Insert into a table

In [176]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5bf3fc4f-ad65-4767-a7a6-d20e9b04a8dc>

# Alpha trench3 App Score

In [177]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'ca_t3_tx_cnt_installments_paid_tot_with_dpd',
       'ca_t3_tx_time_since_last_applied_loan_application_time',
       'ca_t3_tx_last_applied_loan_decisio

In [178]:
d2 = d1[d1['ln_os_type']=='Android']
d2['ln_os_type'].value_counts()

ln_os_type
Android    9420
Name: count, dtype: int64

In [179]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d2.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'c_app_cnt_absence_tag_365d_binned',
        'c_app_cnt_books_and_reference_ever_binned',
        'c_app_cnt_gaming_180d_binned',
        'c_app_cnt_health_and_fitness_ever_binned',
        'c_app_cnt_productivity_ever_binned',
        'c_app_cnt_rated_for_18plus_ever_binned',
        'c_app_last_payday_install_to_apply_days_binned',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('apps_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench3_appscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [180]:
transformeddata = f'Cash_alpha_trench3_appscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench3_appscore_backscore data is: (9420, 13)


In [181]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,1430364,a3954f49-ba98-4dc7-b0bd-140e55748c2f,f08e1185-b7fe-45db-8fb2-db53ba5a53d7,0.371943,2025-09-18T09:05:15.259809,2025-09-18T09:05:15.259809,Cash_alpha_trench3_appscore_backscore,v1,trench alpha beta,137c759f-44f1-4e69-8798-18fd386366ef,2025-09-18T09:05:15.259809,{},"{""c_app_cnt_absence_tag_365d_binned"": ""4.182 \..."
1,2108297,4109130a-0c9b-4754-80e8-3c45060e14ce,12bcc68b-b261-4e91-a31b-766b3de102d7,0.395470,2025-09-18T09:05:15.259809,2025-09-18T09:05:15.259809,Cash_alpha_trench3_appscore_backscore,v1,trench alpha beta,0d1f7a26-71c6-4952-adc8-95bafdd6213f,2025-09-18T09:05:15.259809,{},"{""c_app_cnt_absence_tag_365d_binned"": ""> 6.970..."
2,2484864,7b98aca7-c895-4672-a275-4acac0bdac90,1e31a357-6b4c-4ef3-b583-694675778d04,0.395863,2025-09-18T09:05:15.259809,2025-09-18T09:05:15.259809,Cash_alpha_trench3_appscore_backscore,v1,trench alpha beta,22dad88f-75ed-401d-9f37-0fa353ba8a49,2025-09-18T09:05:15.259809,{},"{""c_app_cnt_absence_tag_365d_binned"": ""4.182 \..."
3,2392508,b1fa0959-b911-4f2c-971a-7faef8a48bde,9443f330-e44c-4629-bbc7-57fe31fb8744,0.399264,2025-09-18T09:05:15.260807,2025-09-18T09:05:15.260807,Cash_alpha_trench3_appscore_backscore,v1,trench alpha beta,abe2311f-3a50-4ca4-a438-03cb670230f9,2025-09-18T09:05:15.260807,{},"{""c_app_cnt_absence_tag_365d_binned"": ""4.182 \..."
4,1965519,4433f45a-ccae-4256-b65b-19a483ce83b0,38d2789c-44fa-468e-85f3-b0cedd6a73a8,0.401831,2025-09-18T09:05:15.260807,2025-09-18T09:05:15.260807,Cash_alpha_trench3_appscore_backscore,v1,trench alpha beta,473cdbf1-4d4a-47be-abdb-686ce94dceec,2025-09-18T09:05:15.260807,{},"{""c_app_cnt_absence_tag_365d_binned"": ""4.182 \..."


In [182]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [183]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench3_appscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_appscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_appscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_appscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_appscore_backscore.joblib


# Insert into a table

In [184]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=65e46f7c-6339-4797-9153-ded816b3f9bf>

# Alpha trench3 Transaction Score

In [185]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'ca_t3_tx_cnt_installments_paid_tot_with_dpd',
       'ca_t3_tx_time_since_last_applied_loan_application_time',
       'ca_t3_tx_last_applied_loan_decisio

In [186]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'ca_t3_tx_cnt_installments_paid_tot_with_dpd',
       'ca_t3_tx_time_since_last_applied_loan_application_time',
       'ca_t3_tx_last_applied_loan_decision',
       'ca_t3_tx_min_age_completed_loans', 'ca_t3_tx_dob_observation_date',
       'ca_t3_tx_cnt_jira_tickets_created_bin', 'ca_t3_tx_max_ever_dpd',
       'ca_t3_tx_amt_cash_in_total', 'ca_t3_tx_last_applied_loan_type_bin',
       'ca_t3_tx_cnt_completed_loans', 'ca_t3_tx_meng_no_of_logins',
       'ca_t3_tx_last_applied_loan_tenor',
       'ca_t3_tx_med_days_bt_cash_out_trans',
       'ca_t3_tx_avg_days_bt_cash_in_trans',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('trx_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench3_transactionscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [187]:
transformeddata = f'Cash_alpha_trench3_transactionscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench3_transactionscore_backscore data is: (16504, 13)


In [188]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,1854550,7aa29106-8b0c-4749-ba35-2bc56713b481,62b3ef53-164a-48d6-88e6-4ac06c383238,0.469502,2025-09-18T09:05:25.398143,2025-09-18T09:05:25.398143,Cash_alpha_trench3_transactionscore_backscore,v1,trench alpha beta,6b1c0b1d-e20c-410f-9372-8200a03dd905,2025-09-18T09:05:25.398143,{},"{""ca_t3_tx_cnt_installments_paid_tot_with_dpd""..."
1,2270274,f3157ede-9e52-409d-a360-a0b8e4a7df9a,3a2cb8ee-6444-44eb-8ae2-2317abed5612,0.530002,2025-09-18T09:05:25.398143,2025-09-18T09:05:25.398143,Cash_alpha_trench3_transactionscore_backscore,v1,trench alpha beta,3e0a85d5-a3c3-4ab0-bcee-7f0b5e8b6f29,2025-09-18T09:05:25.398143,{},"{""ca_t3_tx_cnt_installments_paid_tot_with_dpd""..."
2,1590642,c70c826c-78d6-4dcd-ac11-aae802e26645,fc5debf8-ccc3-4f98-8589-368099178877,0.618469,2025-09-18T09:05:25.399141,2025-09-18T09:05:25.399141,Cash_alpha_trench3_transactionscore_backscore,v1,trench alpha beta,9ac57f47-883c-467b-8405-49854bb8b9a3,2025-09-18T09:05:25.399141,{},"{""ca_t3_tx_cnt_installments_paid_tot_with_dpd""..."
3,2421648,e59cb3e5-0f78-4c81-bfc7-e3d3e28a5dfb,1fae702b-97ce-41fd-acf4-3456a21d8e5a,0.428556,2025-09-18T09:05:25.399141,2025-09-18T09:05:25.399141,Cash_alpha_trench3_transactionscore_backscore,v1,trench alpha beta,def8abfe-d627-490d-9c6e-b369de388d94,2025-09-18T09:05:25.399141,{},"{""ca_t3_tx_cnt_installments_paid_tot_with_dpd""..."
4,1411743,fd9be7a5-f0c9-4110-a45a-fcfe9736d665,1c6ff122-ce5a-445f-979c-140e5395ed0e,0.479491,2025-09-18T09:05:25.399141,2025-09-18T09:05:25.399141,Cash_alpha_trench3_transactionscore_backscore,v1,trench alpha beta,be35d60c-8ee5-4192-a26f-602d2c1e3aa5,2025-09-18T09:05:25.399141,{},"{""ca_t3_tx_cnt_installments_paid_tot_with_dpd""..."


In [189]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [190]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench3_transactionscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_transactionscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_transactionscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_transactionscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_transactionscore_backscore.joblib


# Insert into a table

In [191]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=079a4e29-1e71-4e03-ab6d-0f81bfe869cb>

# Alpha trench3 CIC Score

In [192]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'ca_t3_tx_cnt_installments_paid_tot_with_dpd',
       'ca_t3_tx_time_since_last_applied_loan_application_time',
       'ca_t3_tx_last_applied_loan_decisio

In [193]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
        'ca_cic_max_age_all_contracts_snapshot',
       'ca_cic_ratio_overdue_contracts_to_granted_contracts',
       'ca_cic_ScoreRange', 'ca_cic_ln_loan_level_user_type',
       'ca_cic_has_ever_been_overdue',
       'ca_cic_latest_granted_contract_overdue_flag',
       'ca_cic_ratio_closed_over_new_granted_cnt_24M',
       'ca_cic_ratio_risky_contracts_to_granted_contracts',
       'ca_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'ca_cic_flg_zero_non_granted_ever',
       'ca_cic_Personal_Loans_granted_contracts_amt_24M',
       'ca_cic_CreditAvgCreditLimit', 'ca_cic_flg_zero_granted_ever',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('cic_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench3_cicscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [194]:
transformeddata = f'Cash_alpha_trench3_cicscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench3_cicscore_backscore data is: (16504, 13)


In [195]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,1854550,7aa29106-8b0c-4749-ba35-2bc56713b481,54128d5f-a8dd-4542-879e-351d21c871ae,0.404135,2025-09-18T09:05:39.110531,2025-09-18T09:05:39.110531,Cash_alpha_trench3_cicscore_backscore,v1,trench alpha beta,9ef46dce-74c0-401b-abac-cb7c5c13efe7,2025-09-18T09:05:39.110531,{},"{""ca_cic_max_age_all_contracts_snapshot"": 683...."
1,2270274,f3157ede-9e52-409d-a360-a0b8e4a7df9a,43d7ddef-3a09-44a1-be07-9ff02c1438ba,0.345894,2025-09-18T09:05:39.110531,2025-09-18T09:05:39.110531,Cash_alpha_trench3_cicscore_backscore,v1,trench alpha beta,f304f818-bfc7-41b4-aec0-ab4623897118,2025-09-18T09:05:39.110531,{},"{""ca_cic_max_age_all_contracts_snapshot"": 2382..."
2,1590642,c70c826c-78d6-4dcd-ac11-aae802e26645,23762bbc-0a2e-49c4-8a62-a05486a36dc4,0.416212,2025-09-18T09:05:39.110531,2025-09-18T09:05:39.110531,Cash_alpha_trench3_cicscore_backscore,v1,trench alpha beta,2ec3ab56-ba81-4288-bb1a-5109ea149c04,2025-09-18T09:05:39.110531,{},"{""ca_cic_max_age_all_contracts_snapshot"": 405...."
3,2421648,e59cb3e5-0f78-4c81-bfc7-e3d3e28a5dfb,7a889faf-889b-421f-ba4e-c5974f896c53,0.443384,2025-09-18T09:05:39.111536,2025-09-18T09:05:39.111536,Cash_alpha_trench3_cicscore_backscore,v1,trench alpha beta,21ef7c01-a5b8-43b9-81ed-204c32f83460,2025-09-18T09:05:39.111536,{},"{""ca_cic_max_age_all_contracts_snapshot"": 2663..."
4,1411743,fd9be7a5-f0c9-4110-a45a-fcfe9736d665,7b2b952b-e492-4150-83c3-100bfb9f3a66,0.391682,2025-09-18T09:05:39.111536,2025-09-18T09:05:39.111536,Cash_alpha_trench3_cicscore_backscore,v1,trench alpha beta,539bd320-0721-4ae3-9c6f-99f02193052d,2025-09-18T09:05:39.111536,{},"{""ca_cic_max_age_all_contracts_snapshot"": 1989..."


In [196]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [197]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench3_cicscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_cicscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_cicscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_cicscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_cicscore_backscore.joblib


# Insert into a table

In [198]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=791134d5-ea9d-4804-8869-7645c8e89760>

# Alpha trench3 Stack Score

In [199]:
d1.columns

Index(['customer_id', 'digitalLoanAccountId', 'days_on_book',
       'ln_appln_submit_datetime', 'ln_os_type', 'ln_vas_opted_flag',
       'ln_self_dec_income', 'ln_age', 'ln_source_funds_new_bin',
       'ln_loan_level_user_type', 'ln_industry_new_cat_bin',
       'ln_marital_status', 'ln_doc_type_rolled', 'ln_education_level',
       'ln_ref2_type', 'ln_email_primary_domain', 'ln_province_bin',
       'ln_mature_fspd30_flag', 'ln_fspd30_flag', 'trench_category',
       'ln_loan_type', 'ln_disb_dtime', 'c_app_cnt_absence_tag_365d_binned',
       'c_app_cnt_books_and_reference_ever_binned',
       'c_app_cnt_gaming_180d_binned',
       'c_app_cnt_health_and_fitness_ever_binned',
       'c_app_cnt_productivity_ever_binned',
       'c_app_cnt_rated_for_18plus_ever_binned',
       'c_app_last_payday_install_to_apply_days_binned',
       'ca_t3_tx_cnt_installments_paid_tot_with_dpd',
       'ca_t3_tx_time_since_last_applied_loan_application_time',
       'ca_t3_tx_last_applied_loan_decisio

In [200]:
import pandas as pd
import json
import uuid
from datetime import datetime

def transform_data(output_file_path):
    # Read the input CSV file
    df = d1.copy()
    
    # Create the output DataFrame with the required structure
    output_data = []
    
    for _, row in df.iterrows():
        # Create the calcFeature JSON with all the feature columns
        feature_columns = [
         'trx_score', 'demo_score',
       'apps_score', 'credo_score', 'cic_score',
        ]
        
        calc_feature = {}
        for col in feature_columns:
            if col in row and pd.notna(row[col]):
                # Convert Timestamp objects to string
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
       
        # Get current timestamp
        current_time = datetime.now().isoformat()
        
        # Create the output row
        output_row = {
            "customerId": row['customer_id'],
            "digitalLoanAccountId": row['digitalLoanAccountId'],
            "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
            "prediction": row.get('stack_score', 0),
            "start_time": current_time,
            "end_time": current_time,
            "modelDisplayName":"Cash_alpha_trench3_stackscore_backscore",
            "modelVersionId":"v1",
            "subscription_name": "trench alpha beta",
            "message_id": str(uuid.uuid4()),  # Generate random UUID
            "publish_time": current_time,
            "attributes": "{}",  # Empty JSON object
            "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
        }
        
        output_data.append(output_row)
    
    # Create DataFrame from the output data
    output_df = pd.DataFrame(output_data)
    
    # Save to CSV
    output_df.to_csv(output_file_path, index=False)
    return output_df

# Example usage:
# transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# transform_data(f'{LOCALPATH}/{transformeddata}.csv')

In [201]:
transformeddata = f'Cash_alpha_trench3_stackscore_backscore'
dfd = transform_data(f'{LOCALPATH}/{transformeddata}.csv')
print(f"The shape of the {transformeddata} data is: {dfd.shape}")

The shape of the Cash_alpha_trench3_stackscore_backscore data is: (16504, 13)


In [202]:
dfd.head()

,customerId,digitalLoanAccountId,crifApplicationId,prediction,start_time,end_time,modelDisplayName,modelVersionId,subscription_name,message_id,publish_time,attributes,calcFeature
0,1854550,7aa29106-8b0c-4749-ba35-2bc56713b481,a50fc8b0-1ad1-4fab-b081-9bb6c136f098,0.389749,2025-09-18T09:05:54.645366,2025-09-18T09:05:54.645366,Cash_alpha_trench3_stackscore_backscore,v1,trench alpha beta,c15c5ba5-a5ee-4efa-aace-32aa50d4b796,2025-09-18T09:05:54.645366,{},"{""trx_score"": 0.4695020319078807, ""demo_score""..."
1,2270274,f3157ede-9e52-409d-a360-a0b8e4a7df9a,d9c26dd1-0cbe-43b9-9cae-2a054fd3a605,0.446008,2025-09-18T09:05:54.646368,2025-09-18T09:05:54.646368,Cash_alpha_trench3_stackscore_backscore,v1,trench alpha beta,225eab58-df9e-4a9f-9d73-7194c3d53497,2025-09-18T09:05:54.646368,{},"{""trx_score"": 0.5300016312560699, ""demo_score""..."
2,1590642,c70c826c-78d6-4dcd-ac11-aae802e26645,0fb8dd7d-a275-4082-8541-e93fd3c92468,0.538762,2025-09-18T09:05:54.646368,2025-09-18T09:05:54.646368,Cash_alpha_trench3_stackscore_backscore,v1,trench alpha beta,cdb17897-c44a-41f7-a34f-f004ead65ef0,2025-09-18T09:05:54.646368,{},"{""trx_score"": 0.6184687365499864, ""demo_score""..."
3,2421648,e59cb3e5-0f78-4c81-bfc7-e3d3e28a5dfb,278efe32-a892-4895-b001-673a957ebcd1,0.386965,2025-09-18T09:05:54.646368,2025-09-18T09:05:54.646368,Cash_alpha_trench3_stackscore_backscore,v1,trench alpha beta,b038a9cb-8da7-451e-a2c4-41428fb22f08,2025-09-18T09:05:54.646368,{},"{""trx_score"": 0.4285556491269646, ""demo_score""..."
4,1411743,fd9be7a5-f0c9-4110-a45a-fcfe9736d665,71b3ad7e-9d0d-4a4b-91ad-53556f3a3775,0.436300,2025-09-18T09:05:54.646368,2025-09-18T09:05:54.646368,Cash_alpha_trench3_stackscore_backscore,v1,trench alpha beta,f9166293-576c-4957-929c-11e9f191f8c7,2025-09-18T09:05:54.646368,{},"{""trx_score"": 0.479490661700878, ""demo_score"":..."


In [203]:
df1 = dfd[['customerId', 'digitalLoanAccountId', 'prediction',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
        'calcFeature'
       ]].copy()

In [204]:
filenames = f'{CURRENT_DATE}_{unique_id}_{transformeddata}'
print(filenames)

results = save_dataframe_multi_format(
     dataframe=df1,
     cloud_path=CLOUDPATH,
     filename=filenames,
     client=client,
     bucket_name=f'{BUCKETNAME}'
 )

20250918_134933f53c90_Cash_alpha_trench3_stackscore_backscore
All files saved successfully!
CSV: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_stackscore_backscore.csv
PICKLE: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_stackscore_backscore.pkl
PARQUET: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_stackscore_backscore.parquet
JOBLIB: gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Model_Monitoring/Model_Tables/20250918_134933f53c90_Cash_alpha_trench3_stackscore_backscore.joblib


# Insert into a table

In [205]:
# Upload to BigQuery
table_id = f"prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data"
print(table_id)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(df1, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


prj-prod-dataplatform.dap_ds_poweruser_playground.temp_model_transformed_data


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a3599e99-cfd4-4120-a678-32586bd93d1e>

In [206]:
sq = """select modelDisplayName, count(distinct digitalLoanAccountId)cnt from dap_ds_poweruser_playground.temp_model_transformed_data group by 1 order by 1;"""

d2 = client.query(sq).to_dataframe()
d2

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,modelDisplayName,cnt
0,Cash_alpha_trench1_appscorescore_backscore,28364
1,Cash_alpha_trench1_cicscore_backscore,62044
2,Cash_alpha_trench1_demoscore_backscore,62044
3,Cash_alpha_trench1_stackscore_backscore,62044
4,Cash_alpha_trench2_appscore_backscore,22572
5,Cash_alpha_trench2_cicscore_backscore,39651
6,Cash_alpha_trench2_demoscore_backscore,39651
7,Cash_alpha_trench2_stackscore_backscore,39651
8,Cash_alpha_trench2_transactionscore_backscore,39651
9,Cash_alpha_trench3_appscore_backscore,9420


# Merged final table

In [207]:
schema3 = 'dap_ds_poweruser_playground'
tab1 = 'temp_final_model_transformed_data'

In [208]:
sq = f"""
create or replace table {schema3}.{tab1} as
SELECT cast(customerId as numeric)customerId, digitalLoanAccountId, cast(prediction as string)prediction, start_time, end_time  
, modelDisplayName
, modelVersionId
, calcFeature
, 'ml_model_run_details' source
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details` 
qualify row_number() over(partition by customerId ,digitalLoanAccountId, modelDisplayName order by start_time
desc) = 1

union all 

select customerId, digitalLoanAccountId, cast(prediction as string)prediction, datetime(start_time)start_time, datetime(end_time) end_time 
, modelDisplayName
, modelVersionId
, calcFeature 
, 'Manual_Backscore_tables' source
from dap_ds_poweruser_playground.temp_model_transformed_data
;
"""
job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table {schema3}.{tab1} created successfully')



Table dap_ds_poweruser_playground.temp_final_model_transformed_data created successfully


In [209]:
sq = """select modelDisplayName, source, count(distinct digitalLoanAccountId) from dap_ds_poweruser_playground.temp_final_model_transformed_data
group by 1,2
order by 1,2"""

d3 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
d3

Job ID a2928d72-3bf3-4e4f-976d-057fba3f7a9d successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,modelDisplayName,source,f0_
0,Alpha - IncomeEstimationModel,ml_model_run_details,48484
1,Alpha - CIC-SIL-Model,ml_model_run_details,48484
2,Alpha - StackingModel,ml_model_run_details,48484
3,Beta - AppsScoreModel,ml_model_run_details,68464
4,Beta - DemoScoreModel,ml_model_run_details,77380
5,Beta - IncomeEstimationModel,ml_model_run_details,77380
6,Beta - StackScoreModel,ml_model_run_details,77380
7,Cash_alpha_trench1_appscorescore_backscore,Manual_Backscore_tables,28364
8,Cash_alpha_trench1_cicscore_backscore,Manual_Backscore_tables,62044
9,Cash_alpha_trench1_demoscore_backscore,Manual_Backscore_tables,62044


In [210]:
# d1.to_csv(fr"{LOCALPATH}\{CURRENT_DATE}_{unique_id}_{transformeddata}.csv", index = False)
# d1.to_parquet(fr"{LOCALPATH}\{CURRENT_DATE}_{unique_id}_{transformeddata}.parquet")
# d1.to_pickle(fr"{LOCALPATH}\{CURRENT_DATE}_{unique_id}_{transformeddata}.pkl")
# joblib.dump(d1, f"{LOCALPATH}\{CURRENT_DATE}_{unique_id}_{transformeddata}.joblib")